# Win Place Show - a Lambda School Build Week project

Link to Technical Design document: https://docs.google.com/document/d/1JNTuS_4dzkCbfm3G0gWB0jbJpnh9AoWVaZPOKvONQOg/edit

In [3]:
# Initially, this notebook will be unorganized and for brainstorming
# Later, may organize for formal presentation

import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [3]:
# Load kaggle dataset from Luke Byrne

horses_df = pd.read_csv('horses.csv')
odds_exotics_df = pd.read_csv('odds_exotics.csv')

C:\Users\jhump\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (25,26,32,33,38,51,52,53,54,56,57,58,59,61,62) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
odds_exotics_df.head()

,id,market_id,quinella,exacta,trifecta,first_four,created_at,updated_at
0,9479,34342,2.4,3.4,15.6,68.5,2018-07-12 04:07:06.832818,2018-07-12 04:07:06.832818
1,9478,34334,12.0,24.4,120.9,732.9,2018-07-12 04:07:06.642895,2018-07-12 04:07:06.642895
2,9477,34326,135.4,227.5,1927.0,6528.4,2018-07-12 04:07:06.395469,2018-07-12 04:07:06.395469
3,9476,34318,25.3,66.4,237.7,959.4,2018-07-12 04:07:06.096469,2018-07-12 04:07:06.096469
4,9475,34311,21.3,48.6,131.8,430.3,2018-07-12 04:07:05.909905,2018-07-12 04:07:05.909905


In [4]:
horses_df.head()

,previous_margin,position,position_again,position_two,margin,bf_odds,bf_odds_two_mins_out,bf_odds_all,vic_tote,vic_tote_two_mins_out,...,class_stronger_places,first_up_starts,first_up_wins,first_up_places,second_up_starts,second_up_wins,second_up_places,track_distance_starts,track_distance_wins,track_distance_places
0,6.8,1,1.0,1.0,0.0,2.88,3.00,"{3.25,3.05,3.0,2.92,2.88}",2.3,3.5,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,16.6,0,2.0,2.0,NaN,15.00,18.00,"{18.5,11.0,18.0,17.5,15.0}",11.2,11.7,...,0.0,4.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,NaN,0,3.0,3.0,NaN,95.00,100.00,"{60.0,90.0,100.0,140.0,95.0}",46.4,37.1,...,0.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
3,NaN,0,NaN,4.0,NaN,20.00,17.00,"{18.0,15.5,17.0,20.0,20.0}",13.2,14.2,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,NaN,0,NaN,5.0,NaN,2.74,2.68,"{2.66,2.72,2.68,2.86,2.74}",2.6,3.2,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
horses_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221680 entries, 0 to 221679
Columns: 108 entries, previous_margin to track_distance_places
dtypes: bool(10), float64(69), int64(5), object(24)
memory usage: 167.9+ MB


In [6]:
object_cols = horses_df.select_dtypes(include='object')

In [7]:
[col for col in object_cols]

['bf_odds_all',
 'vic_tote_all',
 'nsw_tote_all',
 'country_code',
 'venue_name',
 'date',
 'market_name',
 'condition',
 'weather',
 'name',
 'runner_name_uuid',
 'blinkers',
 'emergency',
 'last_five_starts',
 'sex',
 'sire',
 'dam',
 'colour',
 'jockey',
 'jockey_sex',
 'trainer',
 'form_comment',
 'last_twenty_starts',
 'class_level']

In [8]:
horses_df.bf_odds_all.head()  # dictionary of 5 float values
horses_df.vic_tote_all.head()  # also a dictionary of 5 float values 
horses_df.nsw_tote_all.head()  # another dict of 5 float values
horses_df.date.head()  # replace "-" with "" and convert type to int
horses_df.condition.head()  # "HVY9" in each row of head
horses_df.condition.value_counts()

GOOD4    81874
SOFT5    29009
GOOD3    28712
SOFT6    15648
SOFT7    12246
HVY8     11269
AWT      10281
HVY10     4876
HVY9      4165
GOOD      1023
FIRM2      384
HEAVY        5
Name: condition, dtype: int64

In [9]:
# write a dict for mapping condition by frequency to integer
condition_dict = {'GOOD4': 1, 'SOFT5': 2, 'GOOD3': 3, 'SOFT6': 4,
                  'SOFT7': 5, 'HVY8': 6, 'AWT': 7, 'HVY10': 8,
                  'HVY9': 9, 'GOOD': 10, 'FIRM2': 11, 'HEAVY': 12}

In [10]:
horses_df.weather.value_counts()

FINE     136341
OCAST     46752
SHWRY     16478
Name: weather, dtype: int64

In [11]:
# write a dict for mapping weather by frequency to integer
weather_dict = {'FINE': 1, 'OCAST': 2, 'SHWRY': 3}

In [12]:
# blinkers, emergency, last_five_starts, sex, sire, dam, jockey_sex, last_twenty_starts, class_level
horses_df.blinkers.value_counts()  # boolean, so categorical encode or convert type
horses_df.emergency.value_counts()  # boolean, same
horses_df.last_five_starts.value_counts()  # binary values? length is 83,360, so drop?
horses_df.sex.value_counts()

Gelding    111158
Mare        51412
Filly       25732
Colt         4908
Horse        1629
Unknown        50
Name: sex, dtype: int64

In [13]:
# write a dict to map sex type frequency to integer
horse_sex_dict = {'Gelding': 1, 'Mare': 2, 'Filly': 3, 'Colt': 4,
                  'Horse': 5, 'Unknown': 6}

In [14]:
horses_df.sire.value_counts()[:5]

SEBRING                 2300
SNITZEL                 2075
WRITTEN TYCOON          2010
HIGH CHAPARRAL (IRE)    1935
CASINO PRINCE           1808
Name: sire, dtype: int64

In [15]:
horses_df.sire.value_counts()[-5:]

FRENCH DEPUTY (USA)    1
JULIO REX              1
HELIOSTATIC (IRE)      1
OVERBURY (IRE)         1
COCKNEY REBEL (IRE)    1
Name: sire, dtype: int64

In [16]:
horses_df.dam.value_counts()[:5]

LUST FOR DUST     62
EXPLICIT VIXEN    61
EXALTED BLAZE     57
STARNET           56
CUBAN GIRL        56
Name: dam, dtype: int64

In [17]:
horses_df.jockey_sex.value_counts()  # male/female

Male      152626
Female     42263
Name: jockey_sex, dtype: int64

In [18]:
jockey_sex_dict = {'Male': 1, 'Female': 2}

In [19]:
horses_df.last_twenty_starts.head()  # binary? drop

0              f3
1       f0x6x80x2
2    f68477579x00
3              f0
4             f4x
Name: last_twenty_starts, dtype: object

In [20]:
horses_df.class_level.value_counts()  # "eq", "up", and "dn"

eq    112686
up     44937
dn     37015
Name: class_level, dtype: int64

In [21]:
# yep, another dic
class_dict = {'eq': 1, 'up': 2, 'dn': 3}

In [90]:
horses_df_wking = horses_df.copy()

In [103]:
# Start feature engineering
# drop multiple columns
horses_df_wking = horses_df_wking.drop(['country_code', 'venue_name', 'market_name',
                                        'runner_name_uuid', 'colour', 'jockey',
                                        'trainer', 'form_comment', 'last_five_starts',
                                        'last_twenty_starts'], axis=1)

In [24]:
# write a function to apply to 'bf_odds_all', 'vic_tote_all', and 'nsw_tote_all'
# that will convert dict to list then get mean of the 5 floats in the sequence

# something like-- lambda x: mean(list(x)) --didn't work
# df['col_3'] = df[['col_1','col_2']].apply(lambda x: f(*x), axis=1) -- from SO
from statistics import mean

In [91]:
# thus, drop those 3 columns
horses_df_wking = horses_df_wking.drop(['bf_odds_all', 'vic_tote_all', 'nsw_tote_all'], axis=1)

In [92]:
# need baseline, so, for now, will drop 'name', 'sire', and 'dam'
horses_df_wking = horses_df_wking.drop(['name', 'sire', 'dam'], axis=1)

In [93]:
# engineer date from object type to int
horses_df_wking.date = horses_df_wking.date.str.replace('-', '').astype(int)

In [94]:
# map condition, weather, horse_sex, jockey_sex, class respective dicts
horses_df_wking.condition = horses_df_wking.condition.replace(condition_dict)

In [95]:
horses_df_wking.weather = horses_df_wking.weather.replace(weather_dict)

In [96]:
horses_df_wking.sex = horses_df_wking.sex.replace(horse_sex_dict)

In [97]:
horses_df_wking.jockey_sex = horses_df_wking.jockey_sex.replace(jockey_sex_dict)

In [98]:
horses_df_wking.class_level = horses_df_wking.class_level.replace(class_dict)

In [104]:
[col for col in horses_df_wking.select_dtypes(include='object')]

[]

In [100]:
horses_df_wking.blinkers = horses_df_wking.blinkers.astype('bool')

In [101]:
horses_df_wking.emergency = horses_df_wking.emergency.astype('bool')

In [36]:
horses_df_wking.columns

Index(['previous_margin', 'position', 'position_again', 'position_two',
       'margin', 'bf_odds', 'bf_odds_two_mins_out', 'vic_tote',
       'vic_tote_two_mins_out', 'nsw_tote', 'nsw_tote_two_mins_out',
       'nsw_odds', 'betfair_slope', 'vic_tote_slope', 'nsw_tote_slope',
       'nsw_odds_slope', 'market_id', 'date', 'race_number', 'condition',
       'weather', 'runner_id', 'number', 'barrier', 'blinkers', 'emergency',
       'dfs_form_rating', 'tech_form_rating', 'total_rating_points',
       'handicap_weight', 'penalty', 'favourite_odds_win',
       'favourite_tote_win', 'tip_pundit_win', 'tip_rating_win',
       'tip_12_months_win', 'tip_recent_win', 'tip_distance_win',
       'tip_class_win', 'tip_time_win', 'tip_overall_win', 'prize_money',
       'sex', 'age', 'jockey_sex', 'form_comment_sentiment', 'class_level',
       'field_strength', 'days_since_last_run', 'runs_since_spell',
       'overall_starts', 'overall_wins', 'overall_places', 'track_starts',
       'track_wins',

In [105]:
horses_df_wking.shape

(221680, 92)

In [106]:
horses_df_wking.isna().sum()

previous_margin           89164
position                      0
position_again           158461
position_two               4518
margin                    67938
bf_odds                       0
bf_odds_two_mins_out        192
vic_tote                      0
vic_tote_two_mins_out       192
nsw_tote                      0
nsw_tote_two_mins_out       192
nsw_odds                      0
betfair_slope                61
vic_tote_slope               61
nsw_tote_slope               61
nsw_odds_slope               61
market_id                     0
date                          0
race_number                   0
condition                 22188
weather                   22109
runner_id                     0
number                        0
barrier                   26289
blinkers                      0
emergency                     0
dfs_form_rating           26299
tech_form_rating          26289
total_rating_points       26299
handicap_weight           26289
                          ...  
dead_sta

In [107]:
# drop 'previous_margin', 'position_again', and 'margin'
horses_df_wking = horses_df_wking.drop(['previous_margin', 'position_again',
                                        'margin'], axis=1)

In [109]:
# replace nan's with respective column modes
for col in horses_df_wking.columns:
    horses_df_wking[col].fillna(horses_df_wking[col].mode()[0], inplace=True)

In [110]:
horses_df_wking.isna().sum().sum()

0

In [111]:
# Define X and y data
X = horses_df_wking.drop('position', axis=1)
y = horses_df_wking['position']

In [112]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.333)

In [113]:
# 
#
count = {k:0 for k in set(y)}
for yy in y:
    count[yy] += 1
sorted(count.items(), key=lambda i: i[1])

[(1, 22234), (0, 199446)]

In [114]:
# Okay, should now be able to apply model. Am going straight to ExtraTrees Classifier

etc = ExtraTreesClassifier(n_estimators=10, max_depth=None,
                           min_samples_split=2, random_state=7)

In [115]:
etc.fit(X_train, y_train)

ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=7, verbose=0, warm_start=False)

In [116]:
etc.score(X_test, y_test)

0.9612706583581685

In [ ]:
horses_df_wking.to_csv('horses_alpha_clean.csv', index=False)

In [ ]:
from tpot import TPOTClassifier

In [ ]:
tpot = TPOTClassifier(generations=5, population_size=20, verbosity=2)
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))

In [57]:
from sklearn.ensemble import RandomForestClassifier

In [58]:
rfc = RandomForestClassifier(n_estimators=100, random_state=7)

In [117]:
rfc.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=7, verbose=0, warm_start=False)

In [60]:
rfc.score(X_test, y_test)

0.9549715524248171

In [61]:
import keras

Using TensorFlow backend.


In [81]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [87]:
model = Sequential()
model.add(Dense(1, input_dim=88, activation='relu'))
model.add(Dense(64, activation='relu'))
# model.add(Dropout(rate=0.1))
# model.add(Dense(24, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='mean_absolute_error', optimizer='sgd', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=50)
scores = model.evaluate(X_test, y_test)
print(f"{model.metrics[0]}: {scores[1]*100}")

Epoch 1/50
147860/147860 [==============================] - 18s 121us/step - loss: 1.8299 - acc: 0.4157
Epoch 2/50
147860/147860 [==============================] - 17s 113us/step - loss: 1.8299 - acc: 0.4157
Epoch 3/50
147860/147860 [==============================] - ETA: 0s - loss: 1.8295 - acc: 0.415 - 17s 116us/step - loss: 1.8299 - acc: 0.4157
Epoch 4/50
147860/147860 [==============================] - 17s 118us/step - loss: 1.8299 - acc: 0.4157
Epoch 5/50
147860/147860 [==============================] - 16s 107us/step - loss: 1.8299 - acc: 0.4157
Epoch 6/50
147860/147860 [==============================] - 15s 103us/step - loss: 1.8299 - acc: 0.4157
Epoch 7/50
147860/147860 [==============================] - 18s 119us/step - loss: 1.8299 - acc: 0.4157
Epoch 8/50
147860/147860 [==============================] - 18s 123us/step - loss: 1.8299 - acc: 0.4157
Epoch 9/50
147860/147860 [==============================] - 18s 122us/step - loss: 1.8299 - acc: 0.4157
Epoch 10/50
147860/147860 

In [ ]:
# try a different model
batch_size = 80  # = 20 gave same 41.8775% accuracy as baseline model

model1 = Sequential()
model1.add(Dense(1, input_dim=88, activation='relu'))
model1.add(Dense(64, activation='relu'))
# model.add(Dropout(rate=0.1))
# model.add(Dense(24, activation='relu'))
model1.add(Dense(1, activation='sigmoid'))
model1.compile(loss='mean_absolute_error', optimizer='sgd', metrics=['accuracy'])
model1.fit(X_train, y_train, epochs=50, batch_size=batch_size)
scores = model1.evaluate(X_test, y_test)
print(f"{model1.metrics[0]}: {scores[1]*100}")

In [8]:
# Bring in Railbird 2019 Kentucky Derby prep race data
# credit to Jessica Chapel [@railbird]
import requests

In [9]:
# Load 2019 prep races data
!curl https://docs.google.com/spreadsheets/d/1q3j7Y8pq_tXJMt_YU-nel9PZY1Jrwell_YsF2bp8D6Y/pub#

<!DOCTYPE html><html><head><title>Railbird: 2019 Kentucky Derby Prep Races</title><meta name="viewport" content="target-densitydpi=device-dpi,user-scalable=1,minimum-scale=1,maximum-scale=2.5,initial-scale=1,width=device-width"><link href='/static/spreadsheets2/client/css/2600819229-waffle_k_ltr.css' type='text/css' rel='stylesheet'><style type="text/css">
        html { overflow: visible; }
        #sheets-viewport { overflow: auto; }
        #sheets-viewport.widget-viewport { overflow: hidden; }
        .grid-container { overflow: visible; background: white;}
        .grid-table-container { overflow: visible; }
        #top-bar {
          background: url("//ssl.gstatic.com/docs/spreadsheets/publishheader.png") repeat-x bottom;
          margin: 0;
          overflow: hidden;
        }
        #top-bar {
          border-bottom: 1px solid #ccc;
          padding: 6px 6px 0;
        }
        #doc-title { padding-bottom: 5px; }
        #doc-title .name { font-size: 15px; }
        #sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  2893    0  2893    0     0   3221      0 --:--:-- --:--:-- --:--:--  3218
100  410k    0  410k    0     0   326k      0 --:--:--  0:00:01 --:--:--  326k


In [10]:
url = 'https://docs.google.com/spreadsheets/d/1q3j7Y8pq_tXJMt_YU-nel9PZY1Jrwell_YsF2bp8D6Y/pub#'

In [11]:
preps_2019 = pd.read_html(url)

In [12]:
type(preps_2019)

list

In [204]:
preps_2019[0]

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21
0,1,DATE,TRK,GRD,DIST,POINTS (LEADERS),RACE (CHART),REPLAY,WINNER,BSF,...,MRGN,ODDS,JOCKEY,TRAINER,SIRE,BR-SIRE,PLACED,NaN,NaN,NaN
1,2,01-01-2019,AQU,---,8,10-4-2-1 (PDF),Jerome Stakes (PDF),Video,Mind Control,88,...,1 1/2,1.50**,John Velazquez,Gregory Sacco,Stay Thirsty,Lightnin N Thunder,"Our Braintrust, Family Biz, Gates of Dawn",NaN,NaN,NaN
2,3,01-05-2019,SA,G3,8,10-4-2-1 (PDF),Sham Stakes (PDF),Video,Gunmetal Gray,82,...,1,3.70,Mike Smith,Jerry Hollendorfer,Exchange Rate,Include,"Sueno, Much Better, Gray Magician",NaN,NaN,NaN
3,4,01-19-2019,FG,G3,8*,10-4-2-1 (PDF),Lecomte Stakes (PDF),Video,War of Will,94,...,4,1.60**,Tyler Gaffalione,Mark Casse,War Front,Sadler's Wells,"Hog Creek Hustle, Manny Wah, Wicked Indeed",NaN,NaN,NaN
4,5,01-25-2019,OP,---,8,10-4-2-1 (PDF),Smarty Jones Stakes (PDF),Video,Gray Attempt,84,...,Neck,2.20,Shaun Bridgmohan,William Fires,Graydar,Consolidator,"Long Range Teddy, Boldor, Six Shooter",NaN,NaN,NaN
5,6,02-02-2019,AQU,G3,9,10-4-2-1 (PDF),Withers Stakes (PDF),Video,Tax,96,...,Head,2.10**,Junior Alvarado,Danny Gargan,Arch,Giant's Causeway,"Not That Brady, Our Braintrust, Sir Winston",NaN,NaN,NaN
6,7,02-02-2019,GP,G3,8.5,10-4-2-1 (PDF),Holy Bull Stakes (PDF),Video,Harvey Wallbanger,85,...,1,29.60,Brian Hernandez,Kenny McPeek,Congrats,Distorted Humor,"Everfast, Maximus Mischief, Epic Dreamer",NaN,NaN,NaN
7,8,02-02-2019,SA,G3,8.5,10-4-2-1 (PDF),Robert B. Lewis (PDF),Video,Mucho Gusto,90,...,4 3/4,0.60**,Joe Talamo,Bob Baffert,Mucho Macho Man,Giant's Causeway,"Gunmetal Gray, Easy Shot, Magnificent McCool",NaN,NaN,NaN
8,9,02-09-2019,TAM,G3,8.5,10-4-2-1 (PDF),Sam F. Davis (PDF),Video,Well Defined,93,...,2 3/4,7.00,Pablo Morales,Kathleen O'Connell,With Distinction,Medaglia d'Oro,"Kentucky Wildcat, So Alive, Counter Offer",NaN,NaN,NaN
9,10,02-16-2019,GG,---,9,10-4-2-1 (PDF),El Camino Real (PDF),Video,Anothertwistafate,94,...,7,1.20**,Juan Hernandez,Blaine Wright,Scat Daddy,First Defence,"Kingly, More Ice, Angelo's Pride",NaN,NaN,NaN


In [13]:
preps_2019_df = pd.DataFrame(preps_2019[0])

In [14]:
preps_2019_df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21
0,1,DATE,TRK,GRD,DIST,POINTS (LEADERS),RACE (CHART),REPLAY,WINNER,BSF,...,MRGN,ODDS,JOCKEY,TRAINER,SIRE,BR-SIRE,PLACED,NaN,NaN,NaN
1,2,01-01-2019,AQU,---,8,10-4-2-1 (PDF),Jerome Stakes (PDF),Video,Mind Control,88,...,1 1/2,1.50**,John Velazquez,Gregory Sacco,Stay Thirsty,Lightnin N Thunder,"Our Braintrust, Family Biz, Gates of Dawn",NaN,NaN,NaN
2,3,01-05-2019,SA,G3,8,10-4-2-1 (PDF),Sham Stakes (PDF),Video,Gunmetal Gray,82,...,1,3.70,Mike Smith,Jerry Hollendorfer,Exchange Rate,Include,"Sueno, Much Better, Gray Magician",NaN,NaN,NaN
3,4,01-19-2019,FG,G3,8*,10-4-2-1 (PDF),Lecomte Stakes (PDF),Video,War of Will,94,...,4,1.60**,Tyler Gaffalione,Mark Casse,War Front,Sadler's Wells,"Hog Creek Hustle, Manny Wah, Wicked Indeed",NaN,NaN,NaN
4,5,01-25-2019,OP,---,8,10-4-2-1 (PDF),Smarty Jones Stakes (PDF),Video,Gray Attempt,84,...,Neck,2.20,Shaun Bridgmohan,William Fires,Graydar,Consolidator,"Long Range Teddy, Boldor, Six Shooter",NaN,NaN,NaN


In [15]:
preps_2019_df.rename(columns=preps_2019_df.iloc[0]).drop(preps_2019_df.index[0])

,1,DATE,TRK,GRD,DIST,POINTS (LEADERS),RACE (CHART),REPLAY,WINNER,BSF,...,MRGN,ODDS,JOCKEY,TRAINER,SIRE,BR-SIRE,PLACED,nan,nan,nan
1,2,01-01-2019,AQU,---,8,10-4-2-1 (PDF),Jerome Stakes (PDF),Video,Mind Control,88,...,1 1/2,1.50**,John Velazquez,Gregory Sacco,Stay Thirsty,Lightnin N Thunder,"Our Braintrust, Family Biz, Gates of Dawn",NaN,NaN,NaN
2,3,01-05-2019,SA,G3,8,10-4-2-1 (PDF),Sham Stakes (PDF),Video,Gunmetal Gray,82,...,1,3.70,Mike Smith,Jerry Hollendorfer,Exchange Rate,Include,"Sueno, Much Better, Gray Magician",NaN,NaN,NaN
3,4,01-19-2019,FG,G3,8*,10-4-2-1 (PDF),Lecomte Stakes (PDF),Video,War of Will,94,...,4,1.60**,Tyler Gaffalione,Mark Casse,War Front,Sadler's Wells,"Hog Creek Hustle, Manny Wah, Wicked Indeed",NaN,NaN,NaN
4,5,01-25-2019,OP,---,8,10-4-2-1 (PDF),Smarty Jones Stakes (PDF),Video,Gray Attempt,84,...,Neck,2.20,Shaun Bridgmohan,William Fires,Graydar,Consolidator,"Long Range Teddy, Boldor, Six Shooter",NaN,NaN,NaN
5,6,02-02-2019,AQU,G3,9,10-4-2-1 (PDF),Withers Stakes (PDF),Video,Tax,96,...,Head,2.10**,Junior Alvarado,Danny Gargan,Arch,Giant's Causeway,"Not That Brady, Our Braintrust, Sir Winston",NaN,NaN,NaN
6,7,02-02-2019,GP,G3,8.5,10-4-2-1 (PDF),Holy Bull Stakes (PDF),Video,Harvey Wallbanger,85,...,1,29.60,Brian Hernandez,Kenny McPeek,Congrats,Distorted Humor,"Everfast, Maximus Mischief, Epic Dreamer",NaN,NaN,NaN
7,8,02-02-2019,SA,G3,8.5,10-4-2-1 (PDF),Robert B. Lewis (PDF),Video,Mucho Gusto,90,...,4 3/4,0.60**,Joe Talamo,Bob Baffert,Mucho Macho Man,Giant's Causeway,"Gunmetal Gray, Easy Shot, Magnificent McCool",NaN,NaN,NaN
8,9,02-09-2019,TAM,G3,8.5,10-4-2-1 (PDF),Sam F. Davis (PDF),Video,Well Defined,93,...,2 3/4,7.00,Pablo Morales,Kathleen O'Connell,With Distinction,Medaglia d'Oro,"Kentucky Wildcat, So Alive, Counter Offer",NaN,NaN,NaN
9,10,02-16-2019,GG,---,9,10-4-2-1 (PDF),El Camino Real (PDF),Video,Anothertwistafate,94,...,7,1.20**,Juan Hernandez,Blaine Wright,Scat Daddy,First Defence,"Kingly, More Ice, Angelo's Pride",NaN,NaN,NaN
10,11,02-16-2019,FG,G2,8.5,50-20-10-5 (PDF),Risen Star Stakes (PDF),Video,War of Will,92,...,2 1/4,1.00**,Tyler Gaffalione,Mark Casse,War Front,Sadler's Wells,"Country House, Roiland, Hog Creek Hustle",NaN,NaN,NaN


In [16]:
preps_2019_df.columns

Index(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
       'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13',
       'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17',
       'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21'],
      dtype='object')

In [17]:
preps_2019_df.drop(['Unnamed: 0', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21'], axis=1)

,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,DATE,TRK,GRD,DIST,POINTS (LEADERS),RACE (CHART),REPLAY,WINNER,BSF,TFUS,TIME,MRGN,ODDS,JOCKEY,TRAINER,SIRE,BR-SIRE,PLACED
1,01-01-2019,AQU,---,8,10-4-2-1 (PDF),Jerome Stakes (PDF),Video,Mind Control,88,107,1:39.06,1 1/2,1.50**,John Velazquez,Gregory Sacco,Stay Thirsty,Lightnin N Thunder,"Our Braintrust, Family Biz, Gates of Dawn"
2,01-05-2019,SA,G3,8,10-4-2-1 (PDF),Sham Stakes (PDF),Video,Gunmetal Gray,82,101,1:38.96,1,3.70,Mike Smith,Jerry Hollendorfer,Exchange Rate,Include,"Sueno, Much Better, Gray Magician"
3,01-19-2019,FG,G3,8*,10-4-2-1 (PDF),Lecomte Stakes (PDF),Video,War of Will,94,110,1:43.44,4,1.60**,Tyler Gaffalione,Mark Casse,War Front,Sadler's Wells,"Hog Creek Hustle, Manny Wah, Wicked Indeed"
4,01-25-2019,OP,---,8,10-4-2-1 (PDF),Smarty Jones Stakes (PDF),Video,Gray Attempt,84,110,1:36.94,Neck,2.20,Shaun Bridgmohan,William Fires,Graydar,Consolidator,"Long Range Teddy, Boldor, Six Shooter"
5,02-02-2019,AQU,G3,9,10-4-2-1 (PDF),Withers Stakes (PDF),Video,Tax,96,118,1:50.23,Head,2.10**,Junior Alvarado,Danny Gargan,Arch,Giant's Causeway,"Not That Brady, Our Braintrust, Sir Winston"
6,02-02-2019,GP,G3,8.5,10-4-2-1 (PDF),Holy Bull Stakes (PDF),Video,Harvey Wallbanger,85,106,1:43.69,1,29.60,Brian Hernandez,Kenny McPeek,Congrats,Distorted Humor,"Everfast, Maximus Mischief, Epic Dreamer"
7,02-02-2019,SA,G3,8.5,10-4-2-1 (PDF),Robert B. Lewis (PDF),Video,Mucho Gusto,90,111,1:41.81,4 3/4,0.60**,Joe Talamo,Bob Baffert,Mucho Macho Man,Giant's Causeway,"Gunmetal Gray, Easy Shot, Magnificent McCool"
8,02-09-2019,TAM,G3,8.5,10-4-2-1 (PDF),Sam F. Davis (PDF),Video,Well Defined,93,114,1:42.70,2 3/4,7.00,Pablo Morales,Kathleen O'Connell,With Distinction,Medaglia d'Oro,"Kentucky Wildcat, So Alive, Counter Offer"
9,02-16-2019,GG,---,9,10-4-2-1 (PDF),El Camino Real (PDF),Video,Anothertwistafate,94,111,1:50.38,7,1.20**,Juan Hernandez,Blaine Wright,Scat Daddy,First Defence,"Kingly, More Ice, Angelo's Pride"


In [18]:
col_names = ['DATE', 'TRK', 'GRD', 'DIST', 'POINTS (LEADERS)', 'RACE (CHART)',
             'REPLAY', 'WINNER', 'BSF', 'TFUS', 'TIME', 'MRGN', 'ODDS',
             'JOCKEY', 'TRAINER', 'SIRE', 'BR-SIRE', 'PLACED',]

In [19]:
preps_2019_df.shape

(33, 22)

In [20]:
preps_2019_df = preps_2019_df.drop(['Unnamed: 0', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21'], axis=1)

In [21]:
preps_2019_df.shape

(33, 18)

In [22]:
preps_2019_df = preps_2019_df.rename(columns=dict(zip(preps_2019_df.columns, col_names)))

In [23]:
preps_2019_df.head()

,DATE,TRK,GRD,DIST,POINTS (LEADERS),RACE (CHART),REPLAY,WINNER,BSF,TFUS,TIME,MRGN,ODDS,JOCKEY,TRAINER,SIRE,BR-SIRE,PLACED
0,DATE,TRK,GRD,DIST,POINTS (LEADERS),RACE (CHART),REPLAY,WINNER,BSF,TFUS,TIME,MRGN,ODDS,JOCKEY,TRAINER,SIRE,BR-SIRE,PLACED
1,01-01-2019,AQU,---,8,10-4-2-1 (PDF),Jerome Stakes (PDF),Video,Mind Control,88,107,1:39.06,1 1/2,1.50**,John Velazquez,Gregory Sacco,Stay Thirsty,Lightnin N Thunder,"Our Braintrust, Family Biz, Gates of Dawn"
2,01-05-2019,SA,G3,8,10-4-2-1 (PDF),Sham Stakes (PDF),Video,Gunmetal Gray,82,101,1:38.96,1,3.70,Mike Smith,Jerry Hollendorfer,Exchange Rate,Include,"Sueno, Much Better, Gray Magician"
3,01-19-2019,FG,G3,8*,10-4-2-1 (PDF),Lecomte Stakes (PDF),Video,War of Will,94,110,1:43.44,4,1.60**,Tyler Gaffalione,Mark Casse,War Front,Sadler's Wells,"Hog Creek Hustle, Manny Wah, Wicked Indeed"
4,01-25-2019,OP,---,8,10-4-2-1 (PDF),Smarty Jones Stakes (PDF),Video,Gray Attempt,84,110,1:36.94,Neck,2.20,Shaun Bridgmohan,William Fires,Graydar,Consolidator,"Long Range Teddy, Boldor, Six Shooter"


In [24]:
preps_2019_df = preps_2019_df.drop(preps_2019_df.index[0])

In [25]:
preps_2019_df.head()

,DATE,TRK,GRD,DIST,POINTS (LEADERS),RACE (CHART),REPLAY,WINNER,BSF,TFUS,TIME,MRGN,ODDS,JOCKEY,TRAINER,SIRE,BR-SIRE,PLACED
1,01-01-2019,AQU,---,8,10-4-2-1 (PDF),Jerome Stakes (PDF),Video,Mind Control,88,107,1:39.06,1 1/2,1.50**,John Velazquez,Gregory Sacco,Stay Thirsty,Lightnin N Thunder,"Our Braintrust, Family Biz, Gates of Dawn"
2,01-05-2019,SA,G3,8,10-4-2-1 (PDF),Sham Stakes (PDF),Video,Gunmetal Gray,82,101,1:38.96,1,3.70,Mike Smith,Jerry Hollendorfer,Exchange Rate,Include,"Sueno, Much Better, Gray Magician"
3,01-19-2019,FG,G3,8*,10-4-2-1 (PDF),Lecomte Stakes (PDF),Video,War of Will,94,110,1:43.44,4,1.60**,Tyler Gaffalione,Mark Casse,War Front,Sadler's Wells,"Hog Creek Hustle, Manny Wah, Wicked Indeed"
4,01-25-2019,OP,---,8,10-4-2-1 (PDF),Smarty Jones Stakes (PDF),Video,Gray Attempt,84,110,1:36.94,Neck,2.20,Shaun Bridgmohan,William Fires,Graydar,Consolidator,"Long Range Teddy, Boldor, Six Shooter"
5,02-02-2019,AQU,G3,9,10-4-2-1 (PDF),Withers Stakes (PDF),Video,Tax,96,118,1:50.23,Head,2.10**,Junior Alvarado,Danny Gargan,Arch,Giant's Causeway,"Not That Brady, Our Braintrust, Sir Winston"


In [26]:
preps_2019_df.shape

(32, 18)

In [27]:
preps_2019_df.tail(6)

,DATE,TRK,GRD,DIST,POINTS (LEADERS),RACE (CHART),REPLAY,WINNER,BSF,TFUS,TIME,MRGN,ODDS,JOCKEY,TRAINER,SIRE,BR-SIRE,PLACED
27,04-13-2019,KEE,G3,8.5,20-8-4-2 (PDF),Lexington Stakes (PDF),Video,Owendale,98,120,1:44.14,1 3/4,12.70,Florent Geroux,Brad Cox,Into Mischief,Bernardini,"Anothertwistafate, Sueno, Knicks Go"
28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,"*1 mile, 40/70 yards / DIST=Race distance in f...","*1 mile, 40/70 yards / DIST=Race distance in f...","*1 mile, 40/70 yards / DIST=Race distance in f...","*1 mile, 40/70 yards / DIST=Race distance in f...","*1 mile, 40/70 yards / DIST=Race distance in f...","*1 mile, 40/70 yards / DIST=Race distance in f...","*1 mile, 40/70 yards / DIST=Race distance in f...","*1 mile, 40/70 yards / DIST=Race distance in f...","*1 mile, 40/70 yards / DIST=Race distance in f...","*1 mile, 40/70 yards / DIST=Race distance in f...","*1 mile, 40/70 yards / DIST=Race distance in f...","*1 mile, 40/70 yards / DIST=Race distance in f...","*1 mile, 40/70 yards / DIST=Race distance in f...","*1 mile, 40/70 yards / DIST=Race distance in f...","*1 mile, 40/70 yards / DIST=Race distance in f...","*1 mile, 40/70 yards / DIST=Race distance in f...","*1 mile, 40/70 yards / DIST=Race distance in f...","*1 mile, 40/70 yards / DIST=Race distance in f..."
30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,2019 Kentucky Derby prep schedule compiled and...,2019 Kentucky Derby prep schedule compiled and...,2019 Kentucky Derby prep schedule compiled and...,2019 Kentucky Derby prep schedule compiled and...,2019 Kentucky Derby prep schedule compiled and...,2019 Kentucky Derby prep schedule compiled and...,2019 Kentucky Derby prep schedule compiled and...,2019 Kentucky Derby prep schedule compiled and...,2019 Kentucky Derby prep schedule compiled and...,2019 Kentucky Derby prep schedule compiled and...,2019 Kentucky Derby prep schedule compiled and...,2019 Kentucky Derby prep schedule compiled and...,2019 Kentucky Derby prep schedule compiled and...,2019 Kentucky Derby prep schedule compiled and...,2019 Kentucky Derby prep schedule compiled and...,2019 Kentucky Derby prep schedule compiled and...,2019 Kentucky Derby prep schedule compiled and...,2019 Kentucky Derby prep schedule compiled and...
32,View the 2003-2011 Kentucky Derby prep schedul...,View the 2003-2011 Kentucky Derby prep schedul...,View the 2003-2011 Kentucky Derby prep schedul...,View the 2003-2011 Kentucky Derby prep schedul...,View the 2003-2011 Kentucky Derby prep schedul...,View the 2003-2011 Kentucky Derby prep schedul...,View the 2003-2011 Kentucky Derby prep schedul...,View the 2003-2011 Kentucky Derby prep schedul...,View the 2003-2011 Kentucky Derby prep schedul...,View the 2003-2011 Kentucky Derby prep schedul...,View the 2003-2011 Kentucky Derby prep schedul...,View the 2003-2011 Kentucky Derby prep schedul...,View the 2003-2011 Kentucky Derby prep schedul...,View the 2003-2011 Kentucky Derby prep schedul...,View the 2003-2011 Kentucky Derby prep schedul...,View the 2003-2011 Kentucky Derby prep schedul...,View the 2003-2011 Kentucky Derby prep schedul...,View the 2003-2011 Kentucky Derby prep schedul...


In [28]:
preps_2019_df = preps_2019_df.drop(preps_2019_df.index[[28,29,30,31]])

In [29]:
preps_2019_df.tail(3)

,DATE,TRK,GRD,DIST,POINTS (LEADERS),RACE (CHART),REPLAY,WINNER,BSF,TFUS,TIME,MRGN,ODDS,JOCKEY,TRAINER,SIRE,BR-SIRE,PLACED
26,04-13-2019,OP,G1,9,100-40-20-10 (PDF),Arkansas Derby (PDF),Video,Omaha Beach,101,121,1:49.91,1,1.70**,Mike Smith,Richard Mandella,War Front,Seeking the Gold,"Improbable, Country House, Laughing Fox"
27,04-13-2019,KEE,G3,8.5,20-8-4-2 (PDF),Lexington Stakes (PDF),Video,Owendale,98,120,1:44.14,1 3/4,12.70,Florent Geroux,Brad Cox,Into Mischief,Bernardini,"Anothertwistafate, Sueno, Knicks Go"
28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
preps_2019_df = preps_2019_df.drop(preps_2019_df.index[27])

In [31]:
preps_2019_df.tail(3)

,DATE,TRK,GRD,DIST,POINTS (LEADERS),RACE (CHART),REPLAY,WINNER,BSF,TFUS,TIME,MRGN,ODDS,JOCKEY,TRAINER,SIRE,BR-SIRE,PLACED
25,04-06-2019,SA,G1,9,100-40-20-10 (PDF),Santa Anita Derby (PDF),Video,Roadster,98,113,1:51.28,1/2,3.10,Mike Smith,Bob Baffert,Quality Road,Silver Ghost,"Game Winner, Instagrand, Nolo Contesto"
26,04-13-2019,OP,G1,9,100-40-20-10 (PDF),Arkansas Derby (PDF),Video,Omaha Beach,101,121,1:49.91,1,1.70**,Mike Smith,Richard Mandella,War Front,Seeking the Gold,"Improbable, Country House, Laughing Fox"
27,04-13-2019,KEE,G3,8.5,20-8-4-2 (PDF),Lexington Stakes (PDF),Video,Owendale,98,120,1:44.14,1 3/4,12.70,Florent Geroux,Brad Cox,Into Mischief,Bernardini,"Anothertwistafate, Sueno, Knicks Go"


In [32]:
preps_2019_df.shape

(27, 18)

In [35]:
top_50_sires_2016 = horses_df.sire.value_counts()[:50]  # from Luke Byrne kaggle data
top_50_sires_2016

SEBRING                  2300
SNITZEL                  2075
WRITTEN TYCOON           2010
HIGH CHAPARRAL (IRE)     1935
CASINO PRINCE            1808
FASTNET ROCK             1772
I AM INVINCIBLE          1708
CHOISIR                  1682
NOT A SINGLE DOUBT       1674
LONHRO                   1604
BLACKFRIARS              1574
MAGNUS                   1459
STAR WITNESS             1456
NICCONI                  1434
STREET CRY (IRE)         1430
SNIPPETSON               1420
BEL ESPRIT               1401
EXCEED AND EXCEL         1387
REWARD FOR EFFORT        1340
MAGIC ALBERT             1338
HARD SPUN (USA)          1331
DENMAN                   1321
STRATUM                  1292
COMMANDS                 1288
ARTIE SCHILLER (USA)     1274
DANE SHADOW              1195
HINCHINBROOK             1179
AL MAHER                 1177
ROTHESAY                 1165
SMART MISSILE            1164
NORTHERN METEOR          1134
RESET                    1126
HUSSON (ARG)             1096
SAVABEEL  

In [34]:
preps_2019_df.SIRE.values

array(['Stay Thirsty', 'Exchange Rate', 'War Front', 'Graydar', 'Arch',
       'Congrats', 'Mucho Macho Man', 'With Distinction', 'Scat Daddy',
       'War Front', 'Super Saver', 'Noble Mission', 'Daaher', 'Tapit',
       'Big Brown', nan, 'Take Charge Indy', 'War Front', 'Goldencents',
       'First Samurai', 'Point of Entry', "New Year's Day", 'Tapit',
       'Candy Ride (ARG)', 'Quality Road', 'War Front', 'Into Mischief'],
      dtype=object)

In [36]:
lower_case_preps_sires = preps_2019_df.SIRE.str.lower()

In [37]:
lower_case_preps_sires

1         stay thirsty
2        exchange rate
3            war front
4              graydar
5                 arch
6             congrats
7      mucho macho man
8     with distinction
9           scat daddy
10           war front
11         super saver
12       noble mission
13              daaher
14               tapit
15           big brown
16                 NaN
17    take charge indy
18           war front
19         goldencents
20       first samurai
21      point of entry
22      new year's day
23               tapit
24    candy ride (arg)
25        quality road
26           war front
27       into mischief
Name: SIRE, dtype: object

In [39]:
lower_case_horses_2016_top_sires = horses_df.sire.str.lower()[:50]

In [40]:
lower_case_horses_2016_top_sires

0      coronation prancer
1            equiano (fr)
2            oamaru force
3         bradbury's luck
4        redoute's choice
5         california dane
6           dash for cash
7                   krupt
8         hard spun (usa)
9                domesday
10               o'lonhro
11               commands
12                stryker
13    moscow ballet (ire)
14              haradasun
15         congrats (usa)
16            dane shadow
17              bon hoffa
18                danzero
19               shocking
20       stravinsky (usa)
21                sebring
22              red arrow
23           road to rock
24              haradasun
25           tiznow (usa)
26            flying spur
27                shinzig
28           equiano (fr)
29        hard spun (usa)
30     dylan thomas (ire)
31                choisir
32                excites
33             via corone
34                duporth
35           dubleo (usa)
36    ready's image (usa)
37        war trace (usa)
38        ba

In [41]:
[sire for sire in lower_case_preps_sires if sire in lower_case_horses_2016_top_sires]

[]

In [57]:
y2018_arrays = []
y2018_KD = [2018, 'Kentucky Derby', 'Justify', 'Scat Daddy', 'Bob Baffert', 'Good Magic', 'Audible']
y2018_PS = [2018, 'Preakness Stakes', 'Justify', 'Scat Daddy', 'Bob Baffert', 'Bravazo', 'Tenfold']
y2018_BS = [2018, 'Belmont Stakes', 'Justify', 'Scat Daddy', 'Bob Baffert', 'Gronkowski', 'Hofburg']

y2018_arrays = [y2018_KD, y2018_PS, y2018_BS]

In [58]:
y2017_arrays = []
y2017_KD = [2017, 'Kentucky Derby', 'Always Dreaming', 'Bodemeister', 'Todd Pletcher',
            'Looking at Lee', 'Battle of Midway']
y2017_PS = [2017, 'Preakness Stakes','Cloud Computing','Macleans Music','Chad C. Brown',
            'Classic Empire','Senior Investment']
y2017_BS = [2017, 'Belmont Stakes', 'Tapwrit', 'Tapit', 'Todd Pletcher', 'Irish War Cry', 'Patch']

y2017_arrays = [y2017_KD, y2017_PS, y2017_BS]

In [59]:
y2016_arrays = []
y2016_KD = [2016, 'Kentucky Derby', 'Nyquist', 'Uncle Mo', 'Doug O\'Neill', 'Exaggerator', 'Gun Runner']
y2016_PS = [2016, 'Preakness Stakes', 'Exaggerator', 'Curlin', 'Keith Desormeaux', 'Cherry Wine', 'Nyquist']
y2016_BS = [2016, 'Belmont Stakes', 'Creator', 'Tapit', 'Steve Asmussen', 'Destin', 'Lani']

y2016_arrays = [y2016_KD, y2016_PS, y2016_BS]

In [60]:
tc_race_results_3_years = pd.DataFrame(np.array([y2018_KD, y2018_PS, y2018_BS,
                                                 y2017_KD, y2017_PS, y2017_BS,
                                                 y2016_KD, y2016_PS, y2016_BS]),
                                       columns=['Year', 'Race', 'Winner', 'Sire',
                                                'Trainer', '2nd Pl', '3rd Pl'])

In [61]:
tc_race_results_3_years

,Year,Race,Winner,Sire,Trainer,2nd Pl,3rd Pl
0,2018,Kentucky Derby,Justify,Scat Daddy,Bob Baffert,Good Magic,Audible
1,2018,Preakness Stakes,Justify,Scat Daddy,Bob Baffert,Bravazo,Tenfold
2,2018,Belmont Stakes,Justify,Scat Daddy,Bob Baffert,Gronkowski,Hofburg
3,2017,Kentucky Derby,Always Dreaming,Bodemeister,Todd Pletcher,Looking at Lee,Battle of Midway
4,2017,Preakness Stakes,Cloud Computing,Macleans Music,Chad C. Brown,Classic Empire,Senior Investment
5,2017,Belmont Stakes,Tapwrit,Tapit,Todd Pletcher,Irish War Cry,Patch
6,2016,Kentucky Derby,Nyquist,Uncle Mo,Doug O'Neill,Exaggerator,Gun Runner
7,2016,Preakness Stakes,Exaggerator,Curlin,Keith Desormeaux,Cherry Wine,Nyquist
8,2016,Belmont Stakes,Creator,Tapit,Steve Asmussen,Destin,Lani


In [79]:
past_tc_races = tc_race_results_3_years[['Year', 'Race', 'Winner', '2nd Pl', '3rd Pl']]

In [81]:
past_tc_races.groupby(['Year', 'Race'])

In [82]:
past_tc_races

,Year,Race,Winner,2nd Pl,3rd Pl
0,2018,Kentucky Derby,Justify,Good Magic,Audible
1,2018,Preakness Stakes,Justify,Bravazo,Tenfold
2,2018,Belmont Stakes,Justify,Gronkowski,Hofburg
3,2017,Kentucky Derby,Always Dreaming,Looking at Lee,Battle of Midway
4,2017,Preakness Stakes,Cloud Computing,Classic Empire,Senior Investment
5,2017,Belmont Stakes,Tapwrit,Irish War Cry,Patch
6,2016,Kentucky Derby,Nyquist,Exaggerator,Gun Runner
7,2016,Preakness Stakes,Exaggerator,Cherry Wine,Nyquist
8,2016,Belmont Stakes,Creator,Destin,Lani


In [83]:
past_tc_races_json = past_tc_races.to_json()

In [84]:
past_tc_races_json

'{"Year":{"0":"2018","1":"2018","2":"2018","3":"2017","4":"2017","5":"2017","6":"2016","7":"2016","8":"2016"},"Race":{"0":"Kentucky Derby","1":"Preakness Stakes","2":"Belmont Stakes","3":"Kentucky Derby","4":"Preakness Stakes","5":"Belmont Stakes","6":"Kentucky Derby","7":"Preakness Stakes","8":"Belmont Stakes"},"Winner":{"0":"Justify","1":"Justify","2":"Justify","3":"Always Dreaming","4":"Cloud Computing","5":"Tapwrit","6":"Nyquist","7":"Exaggerator","8":"Creator"},"2nd Pl":{"0":"Good Magic","1":"Bravazo","2":"Gronkowski","3":"Looking at Lee","4":"Classic Empire","5":"Irish War Cry","6":"Exaggerator","7":"Cherry Wine","8":"Destin"},"3rd Pl":{"0":"Audible","1":"Tenfold","2":"Hofburg","3":"Battle of Midway","4":"Senior Investment","5":"Patch","6":"Gun Runner","7":"Nyquist","8":"Lani"}}'

In [87]:
import json
past_tc_races_json = json.loads(past_tc_races_json)
print(json.dumps(past_tc_races_json, indent=2, sort_keys=True))

{
  "2nd Pl": {
    "0": "Good Magic",
    "1": "Bravazo",
    "2": "Gronkowski",
    "3": "Looking at Lee",
    "4": "Classic Empire",
    "5": "Irish War Cry",
    "6": "Exaggerator",
    "7": "Cherry Wine",
    "8": "Destin"
  },
  "3rd Pl": {
    "0": "Audible",
    "1": "Tenfold",
    "2": "Hofburg",
    "3": "Battle of Midway",
    "4": "Senior Investment",
    "5": "Patch",
    "6": "Gun Runner",
    "7": "Nyquist",
    "8": "Lani"
  },
  "Race": {
    "0": "Kentucky Derby",
    "1": "Preakness Stakes",
    "2": "Belmont Stakes",
    "3": "Kentucky Derby",
    "4": "Preakness Stakes",
    "5": "Belmont Stakes",
    "6": "Kentucky Derby",
    "7": "Preakness Stakes",
    "8": "Belmont Stakes"
  },
  "Winner": {
    "0": "Justify",
    "1": "Justify",
    "2": "Justify",
    "3": "Always Dreaming",
    "4": "Cloud Computing",
    "5": "Tapwrit",
    "6": "Nyquist",
    "7": "Exaggerator",
    "8": "Creator"
  },
  "Year": {
    "0": "2018",
    "1": "2018",
    "2": "2018",
    "3

In [50]:
tc_previous_sires = tc_race_results_3_years.Sire.str.lower()

In [52]:
set(tc_previous_sires)

{'bodemeister', 'curlin', 'macleans music', 'scat daddy', 'tapit', 'uncle mo'}

In [53]:
[sire for sire in lower_case_preps_sires if sire in set(tc_previous_sires)]

['scat daddy', 'tapit', 'tapit']

In [88]:
preps_2019_df

,DATE,TRK,GRD,DIST,POINTS (LEADERS),RACE (CHART),REPLAY,WINNER,BSF,TFUS,TIME,MRGN,ODDS,JOCKEY,TRAINER,SIRE,BR-SIRE,PLACED
1,01-01-2019,AQU,---,8,10-4-2-1 (PDF),Jerome Stakes (PDF),Video,Mind Control,88,107,1:39.06,1 1/2,1.50**,John Velazquez,Gregory Sacco,Stay Thirsty,Lightnin N Thunder,"Our Braintrust, Family Biz, Gates of Dawn"
2,01-05-2019,SA,G3,8,10-4-2-1 (PDF),Sham Stakes (PDF),Video,Gunmetal Gray,82,101,1:38.96,1,3.70,Mike Smith,Jerry Hollendorfer,Exchange Rate,Include,"Sueno, Much Better, Gray Magician"
3,01-19-2019,FG,G3,8*,10-4-2-1 (PDF),Lecomte Stakes (PDF),Video,War of Will,94,110,1:43.44,4,1.60**,Tyler Gaffalione,Mark Casse,War Front,Sadler's Wells,"Hog Creek Hustle, Manny Wah, Wicked Indeed"
4,01-25-2019,OP,---,8,10-4-2-1 (PDF),Smarty Jones Stakes (PDF),Video,Gray Attempt,84,110,1:36.94,Neck,2.20,Shaun Bridgmohan,William Fires,Graydar,Consolidator,"Long Range Teddy, Boldor, Six Shooter"
5,02-02-2019,AQU,G3,9,10-4-2-1 (PDF),Withers Stakes (PDF),Video,Tax,96,118,1:50.23,Head,2.10**,Junior Alvarado,Danny Gargan,Arch,Giant's Causeway,"Not That Brady, Our Braintrust, Sir Winston"
6,02-02-2019,GP,G3,8.5,10-4-2-1 (PDF),Holy Bull Stakes (PDF),Video,Harvey Wallbanger,85,106,1:43.69,1,29.60,Brian Hernandez,Kenny McPeek,Congrats,Distorted Humor,"Everfast, Maximus Mischief, Epic Dreamer"
7,02-02-2019,SA,G3,8.5,10-4-2-1 (PDF),Robert B. Lewis (PDF),Video,Mucho Gusto,90,111,1:41.81,4 3/4,0.60**,Joe Talamo,Bob Baffert,Mucho Macho Man,Giant's Causeway,"Gunmetal Gray, Easy Shot, Magnificent McCool"
8,02-09-2019,TAM,G3,8.5,10-4-2-1 (PDF),Sam F. Davis (PDF),Video,Well Defined,93,114,1:42.70,2 3/4,7.00,Pablo Morales,Kathleen O'Connell,With Distinction,Medaglia d'Oro,"Kentucky Wildcat, So Alive, Counter Offer"
9,02-16-2019,GG,---,9,10-4-2-1 (PDF),El Camino Real (PDF),Video,Anothertwistafate,94,111,1:50.38,7,1.20**,Juan Hernandez,Blaine Wright,Scat Daddy,First Defence,"Kingly, More Ice, Angelo's Pride"
10,02-16-2019,FG,G2,8.5,50-20-10-5 (PDF),Risen Star Stakes (PDF),Video,War of Will,92,110,1:44.59,2 1/4,1.00**,Tyler Gaffalione,Mark Casse,War Front,Sadler's Wells,"Country House, Roiland, Hog Creek Hustle"


In [72]:
# 2019 Kentucky Derby contenders - cf. view-source:https://www.kentuckyderby.com/horses

tacitus = ['Tacitus', 'William Mott', 'Tapit', 610]
omaha_beach = ['Omaha Beach', 'Richard Mandella', 'War Front', 1050]
vekoma = ['Vekoma', 'George Weaver', 'Candy Ride (ARG)', 747.6]
plus_que_parfait = ['Plus Que Parfait', 'Brendan Walsh', 'Point of Entry', 1540.4]
roadster = ['Roadster', 'Bob Baffert', 'Quality Road', 636]
by_my_standards = ['By My Standards', 'Bret Calhoun', 'Goldencents', 600]
maximum_security = ['Maximum Security', 'Jason Servis', 'New Year\'s Day', 582.8]
game_winner = ['Game Winner', 'Bob Baffert', 'Candy Ride (ARG)', 1810]
code_of_honor = ['Code of Honor', 'Shug McGaughey', 'Noble Mission', 432.07]
haikal = ['Haikal', 'Kiaran McLaughlin', 'Daaher', 322.5]
improbable = ['Improbable', 'Bob Baffert', 'City Zip', 589.52]
war_of_will = ['War of Will', 'Mark Casse', 'War Front', 440.84]
long_range_toddy = ['Long Range Toddy', 'Steve Asmussen', 'Take Charge Indy', 830]
tax = ['Tax', 'Danny Gargan', 'Arch', 307.5]
cutting_humor = ['Cutting Humor', 'Todd Pletcher', 'First Samurai', 462.467]
win_win_win = ['Win Win Win', 'Michael Trombetta', 'Hat Trick (JPN)', 316]
country_house = ['Country House', 'William Mott', 'Lookin At Lucky', 220]
gray_magician = ['Gray Magician', 'Peter Miller', 'Graydar', 526]
spinoff = ['Spinoff', 'Todd Pletcher', 'Hard Spun', 224]
master_fencer_jpn = ['Master Fencer (JPN)', 'Koichi Tsunoda', 'Just a Way', 234.392]

In [73]:
kd_2019_contenders_df = pd.DataFrame(np.array([tacitus, omaha_beach, vekoma, plus_que_parfait,
                                               roadster, by_my_standards, maximum_security,
                                               game_winner, code_of_honor, haikal, improbable,
                                               war_of_will, long_range_toddy, tax, cutting_humor,
                                               win_win_win, country_house, gray_magician,
                                               spinoff, master_fencer_jpn]),
                                     columns=['Horse', 'Trainer', 'Sire', 'Earnings (K USD)'])

In [92]:
kd_2019_contenders_df

,Horse,Trainer,Sire,Earnings (K USD)
0,Tacitus,William Mott,Tapit,610
1,Omaha Beach,Richard Mandella,War Front,1050
2,Vekoma,George Weaver,Candy Ride (ARG),747.6
3,Plus Que Parfait,Brendan Walsh,Point of Entry,1540.4
4,Roadster,Bob Baffert,Quality Road,636
5,By My Standards,Bret Calhoun,Goldencents,600
6,Maximum Security,Jason Servis,New Year's Day,582.8
7,Game Winner,Bob Baffert,Candy Ride (ARG),1810
8,Code of Honor,Shug McGaughey,Noble Mission,432.07
9,Haikal,Kiaran McLaughlin,Daaher,322.5


In [94]:
kd_2019_contenders_csv = kd_2019_contenders_df.to_csv(index=False)
kd_2019_contenders_csv

"Horse,Trainer,Sire,Earnings (K USD)\r\nTacitus,William Mott,Tapit,610\r\nOmaha Beach,Richard Mandella,War Front,1050\r\nVekoma,George Weaver,Candy Ride (ARG),747.6\r\nPlus Que Parfait,Brendan Walsh,Point of Entry,1540.4\r\nRoadster,Bob Baffert,Quality Road,636\r\nBy My Standards,Bret Calhoun,Goldencents,600\r\nMaximum Security,Jason Servis,New Year's Day,582.8\r\nGame Winner,Bob Baffert,Candy Ride (ARG),1810\r\nCode of Honor,Shug McGaughey,Noble Mission,432.07\r\nHaikal,Kiaran McLaughlin,Daaher,322.5\r\nImprobable,Bob Baffert,City Zip,589.52\r\nWar of Will,Mark Casse,War Front,440.84\r\nLong Range Toddy,Steve Asmussen,Take Charge Indy,830\r\nTax,Danny Gargan,Arch,307.5\r\nCutting Humor,Todd Pletcher,First Samurai,462.467\r\nWin Win Win,Michael Trombetta,Hat Trick (JPN),316\r\nCountry House,William Mott,Lookin At Lucky,220\r\nGray Magician,Peter Miller,Graydar,526\r\nSpinoff,Todd Pletcher,Hard Spun,224\r\nMaster Fencer (JPN),Koichi Tsunoda,Just a Way,234.392\r\n"

In [74]:
horses_master_df = pd.read_csv('prep_races_2019_master.txt')

In [29]:
horses_master_df.head()

,Track Name,Race Number,Date,Year,Track Miles,Surface,Age,Weather,Track Condition,Race,Horse Name,Jockey,Weight,M/E,PP,Finish,Money Won,Trainer,Owner,Lengths Behind Winner
0,OAKLAWN PARK,Race 11,April 13,2019.0,One And One Eighth,Dirt,THREE YEAR,Showery,Sloppy (Sealed),Arkansas Derby Grade 1 - Thoroughbred,Omaha Beach,"Smith, Mike",122.0,L,3.0,1.0,"$600,000","Mandella, Richard","Fox Hill Farms, Inc.",0.00
1,OAKLAWN PARK,Race 11,April 13,2019.0,One And One Eighth,Dirt,THREE YEAR,Showery,Sloppy (Sealed),Arkansas Derby Grade 1 - Thoroughbred,Improbable,"Ortiz, Jose",122.0,L b,1.0,2.0,"$200,000","Baffert, Bob","WinStar Farm LLC, China Horse Club Internatio...",1.00
2,OAKLAWN PARK,Race 11,April 13,2019.0,One And One Eighth,Dirt,THREE YEAR,Showery,Sloppy (Sealed),Arkansas Derby Grade 1 - Thoroughbred,Country House,"Rosario, Joel",118.0,L b,8.0,3.0,"$100,000","Mott, William","Shields, Jr., Mrs. J. V., McFadden, Jr., E. J...",6.75
3,OAKLAWN PARK,Race 11,April 13,2019.0,One And One Eighth,Dirt,THREE YEAR,Showery,Sloppy (Sealed),Arkansas Derby Grade 1 - Thoroughbred,Laughing Fox,"Santana, Jr., Ricardo",118.0,L,5.0,4.0,"$50,000","Asmussen, Steven","Lieblong, Alex and JoAnn",7.75
4,OAKLAWN PARK,Race 11,April 13,2019.0,One And One Eighth,Dirt,THREE YEAR,Showery,Sloppy (Sealed),Arkansas Derby Grade 1 - Thoroughbred,Galilean,"Prat, Flavien",122.0,Lf,7.0,5.0,"$30,000","Hollendorfer, Jerry","West Point Thoroughbreds, Barker, Denise, San...",12.00


In [75]:
horses_master_df.Year = 2019
horses_master_df.head()

,Track Name,Race Number,Date,Year,Track Miles,Surface,Age,Weather,Track Condition,Race,Horse Name,Jockey,Weight,M/E,PP,Finish,Money Won,Trainer,Owner,Lengths Behind Winner
0,OAKLAWN PARK,Race 11,April 13,2019,One And One Eighth,Dirt,THREE YEAR,Showery,Sloppy (Sealed),Arkansas Derby Grade 1 - Thoroughbred,Omaha Beach,"Smith, Mike",122.0,L,3.0,1.0,"$600,000","Mandella, Richard","Fox Hill Farms, Inc.",0.00
1,OAKLAWN PARK,Race 11,April 13,2019,One And One Eighth,Dirt,THREE YEAR,Showery,Sloppy (Sealed),Arkansas Derby Grade 1 - Thoroughbred,Improbable,"Ortiz, Jose",122.0,L b,1.0,2.0,"$200,000","Baffert, Bob","WinStar Farm LLC, China Horse Club Internatio...",1.00
2,OAKLAWN PARK,Race 11,April 13,2019,One And One Eighth,Dirt,THREE YEAR,Showery,Sloppy (Sealed),Arkansas Derby Grade 1 - Thoroughbred,Country House,"Rosario, Joel",118.0,L b,8.0,3.0,"$100,000","Mott, William","Shields, Jr., Mrs. J. V., McFadden, Jr., E. J...",6.75
3,OAKLAWN PARK,Race 11,April 13,2019,One And One Eighth,Dirt,THREE YEAR,Showery,Sloppy (Sealed),Arkansas Derby Grade 1 - Thoroughbred,Laughing Fox,"Santana, Jr., Ricardo",118.0,L,5.0,4.0,"$50,000","Asmussen, Steven","Lieblong, Alex and JoAnn",7.75
4,OAKLAWN PARK,Race 11,April 13,2019,One And One Eighth,Dirt,THREE YEAR,Showery,Sloppy (Sealed),Arkansas Derby Grade 1 - Thoroughbred,Galilean,"Prat, Flavien",122.0,Lf,7.0,5.0,"$30,000","Hollendorfer, Jerry","West Point Thoroughbreds, Barker, Denise, San...",12.00


In [76]:
horses_master_procd_df = horses_master_df[['Year', 'Date', 'Race', 'Horse Name',
                                           'Finish', 'Money Won']]

In [77]:
horses_master_procd_df

,Year,Date,Race,Horse Name,Finish,Money Won
0,2019,April 13,Arkansas Derby Grade 1 - Thoroughbred,Omaha Beach,1.0,"$600,000"
1,2019,April 13,Arkansas Derby Grade 1 - Thoroughbred,Improbable,2.0,"$200,000"
2,2019,April 13,Arkansas Derby Grade 1 - Thoroughbred,Country House,3.0,"$100,000"
3,2019,April 13,Arkansas Derby Grade 1 - Thoroughbred,Laughing Fox,4.0,"$50,000"
4,2019,April 13,Arkansas Derby Grade 1 - Thoroughbred,Galilean,5.0,"$30,000"
5,2019,April 13,Arkansas Derby Grade 1 - Thoroughbred,Long Range Toddy,6.0,"$3,334"
6,2019,April 13,Arkansas Derby Grade 1 - Thoroughbred,Jersey Agenda,7.0,"$3,333"
7,2019,April 13,Arkansas Derby Grade 1 - Thoroughbred,Tikhvin Flew,8.0,"$3,333"
8,2019,April 13,Arkansas Derby Grade 1 - Thoroughbred,Gray Attempt,9.0,"$3,333"
9,2019,April 13,Arkansas Derby Grade 1 - Thoroughbred,Six Shooter,10.0,"$3,333"


In [78]:
horses_master_procd_df = horses_master_procd_df.drop(index=[202,203,204,205,206])

In [79]:
horses_master_procd_df.Finish.dtype

dtype('float64')

In [80]:
horses_master_procd_df = horses_master_procd_df[horses_master_procd_df.Finish < 5.0]

In [81]:
horses_master_procd_df.Finish = horses_master_procd_df.Finish.apply(lambda x: int(x))

In [82]:
horses_master_procd_df_for_json = horses_master_procd_df
horses_master_procd_df

,Year,Date,Race,Horse Name,Finish,Money Won
0,2019,April 13,Arkansas Derby Grade 1 - Thoroughbred,Omaha Beach,1,"$600,000"
1,2019,April 13,Arkansas Derby Grade 1 - Thoroughbred,Improbable,2,"$200,000"
2,2019,April 13,Arkansas Derby Grade 1 - Thoroughbred,Country House,3,"$100,000"
3,2019,April 13,Arkansas Derby Grade 1 - Thoroughbred,Laughing Fox,4,"$50,000"
11,2019,April 6,Toyota Blue Grass S. Grade 2 - Thoroughbred,Vekoma,1,"$600,000"
12,2019,April 6,Toyota Blue Grass S. Grade 2 - Thoroughbred,Win Win Win,2,"$200,000"
13,2019,April 6,Toyota Blue Grass S. Grade 2 - Thoroughbred,Signalman,3,"$100,000"
14,2019,April 6,Toyota Blue Grass S. Grade 2 - Thoroughbred,Somelikeithotbrown,4,"$50,000"
25,2019,February 16,El Camino Real Derby Listed - Thoroughbred,Anothertwistafate,1,"$60,000"
26,2019,February 16,El Camino Real Derby Listed - Thoroughbred,Kingly,2,"$20,000"


In [83]:
horses_master_procd_df['Money Won'] = horses_master_procd_df['Money Won'].apply(lambda x: x.replace('$', ''))

In [84]:
horses_master_procd_df['Money Won'] = horses_master_procd_df['Money Won'].apply(lambda x: x.replace(' ', ''))

In [85]:
horses_master_procd_df['Money Won'] = horses_master_procd_df['Money Won'].apply(lambda x: x.replace(',', ''))

In [86]:
horses_master_procd_df['Money Won'] = horses_master_procd_df['Money Won'].astype(int)

In [87]:
horses_master_procd_df['Money Won'] = horses_master_procd_df['Money Won'] / 1000

In [88]:
horses_master_procd_df = horses_master_procd_df.rename(columns={'Money Won': 'Money Won (K USD)'})

In [90]:
horses_master_procd_df.Date = horses_master_procd_df.Date.apply(lambda x: x.replace(' ', ''))

In [91]:
horses_master_procd_df.Date.unique()

array(['April13', 'April6', 'February16', 'March30', 'March2', 'March9',
       'February2', 'March23', 'January19', 'March16', 'February9',
       'January5', 'January25', 'January1'], dtype=object)

In [92]:
date_mapping = {'January1': 1, 'January5': 5, 'January19': 19,
                'January25': 25, 'February2': 33, 'February9': 40,
                'February16': 47, 'March2': 61, 'March9': 68,
                'March16': 75, 'March23': 82, 'March30': 89,
                'April6': 96, 'April13': 103}

In [93]:
horses_master_procd_df.Date = horses_master_procd_df.Date.replace(date_mapping)

In [94]:
horses_master_procd_df

,Year,Date,Race,Horse Name,Finish,Money Won (K USD)
0,2019,103,Arkansas Derby Grade 1 - Thoroughbred,Omaha Beach,1,600.00
1,2019,103,Arkansas Derby Grade 1 - Thoroughbred,Improbable,2,200.00
2,2019,103,Arkansas Derby Grade 1 - Thoroughbred,Country House,3,100.00
3,2019,103,Arkansas Derby Grade 1 - Thoroughbred,Laughing Fox,4,50.00
11,2019,96,Toyota Blue Grass S. Grade 2 - Thoroughbred,Vekoma,1,600.00
12,2019,96,Toyota Blue Grass S. Grade 2 - Thoroughbred,Win Win Win,2,200.00
13,2019,96,Toyota Blue Grass S. Grade 2 - Thoroughbred,Signalman,3,100.00
14,2019,96,Toyota Blue Grass S. Grade 2 - Thoroughbred,Somelikeithotbrown,4,50.00
25,2019,47,El Camino Real Derby Listed - Thoroughbred,Anothertwistafate,1,60.00
26,2019,47,El Camino Real Derby Listed - Thoroughbred,Kingly,2,20.00


In [95]:
horses_master_procd_df = horses_master_procd_df.sort_values(['Date', 'Race'], ascending=[True, True])
horses_master_procd_df

,Year,Date,Race,Horse Name,Finish,Money Won (K USD)
196,2019,1,Jerome S. Listed - Thoroughbred,Mind Control,1,82.50
197,2019,1,Jerome S. Listed - Thoroughbred,Our Braintrust,2,30.00
198,2019,1,Jerome S. Listed - Thoroughbred,Family Biz,3,18.90
199,2019,1,Jerome S. Listed - Thoroughbred,Gates of Dawn,4,9.00
159,2019,5,Sham S. Grade 3 - Thoroughbred,Gunmetal Gray,1,60.00
160,2019,5,Sham S. Grade 3 - Thoroughbred,Sueno,2,20.00
161,2019,5,Sham S. Grade 3 - Thoroughbred,Much Better,3,12.00
162,2019,5,Sham S. Grade 3 - Thoroughbred,Gray Magician,4,6.00
86,2019,19,LeComte S. Grade 3 - Thoroughbred,War of Will,1,120.00
87,2019,19,LeComte S. Grade 3 - Thoroughbred,Hog Creek Hustle,2,40.00


In [96]:
import json
tc_2019_prep_races = json.loads(horses_master_procd_df_for_json.to_json())
print(json.dumps(tc_2019_prep_races, indent=2, sort_keys=True))

{
  "Date": {
    "0": " April 13",
    "1": " April 13",
    "100": " April 13",
    "101": " April 13",
    "108": " February 2",
    "109": " February 2",
    "11": " April 6",
    "110": " February 2",
    "111": " February 2",
    "113": " March 16",
    "114": " March 16",
    "115": " March 16",
    "116": " March 16",
    "12": " April 6",
    "120": " March 16",
    "121": " March 16",
    "122": " March 16",
    "123": " March 16",
    "13": " April 6",
    "130": " February 16",
    "131": " February 16",
    "132": " February 16",
    "133": " February 16",
    "14": " April 6",
    "144": " April 6",
    "145": " April 6",
    "146": " April 6",
    "147": " April 6",
    "150": " February 9",
    "151": " February 9",
    "152": " February 9",
    "153": " February 9",
    "159": " January 5",
    "160": " January 5",
    "161": " January 5",
    "162": " January 5",
    "166": " January 25",
    "167": " January 25",
    "168": " January 25",
    "169": " January 25",
  

In [15]:
## 2018-04-17 Wednesday, Build Week
# need a model asap, but first need to load and feature-engineer data agreed on by DS team

preps_2018_df = pd.read_csv('combined_2018_edit.csv')

In [16]:
preps_2018_df.shape

(233, 36)

In [17]:
pd.set_option('max_columns', 50)

In [18]:
preps_2018_df.head()

,Track Name,Race Number,Date,Track Miles,Surface,Age,Weather,Track Condition,Race,Horse Name,Jockey,Weight,M/E,PP,Finish,Money Won,Trainer,Owner,Lengths Behind Winner,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35
0,OAKLAWN PARK,Race 11,"April 14, 2018",One And One Eighth,Dirt,THREE YEAR,Cloudy,Fast,Arkansas Derby Grade 1 - Thoroughbred,Magnum Moon,"Saez, Luis",122.0,L,6.0,1.0,"$600,000","Pletcher, Todd","Low, Lawana L. and Robert E.",0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,OAKLAWN PARK,Race 11,"April 14, 2018",One And One Eighth,Dirt,THREE YEAR,Cloudy,Fast,Arkansas Derby Grade 1 - Thoroughbred,Quip,"Geroux, Florent",122.0,L,8.0,2.0,"$200,000","Brisset, Rodolphe","WinStar Farm LLC, China Horse Club Internatio...",4.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,OAKLAWN PARK,Race 11,"April 14, 2018",One And One Eighth,Dirt,THREE YEAR,Cloudy,Fast,Arkansas Derby Grade 1 - Thoroughbred,Solomini,"Prat, Flavien",118.0,L b,5.0,3.0,"$100,000","Baffert, Bob","Zayat Stables, LLC, Magnier, Mrs. John, Tabor...",4.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,OAKLAWN PARK,Race 11,"April 14, 2018",One And One Eighth,Dirt,THREE YEAR,Cloudy,Fast,Arkansas Derby Grade 1 - Thoroughbred,Combatant,"Santana, Jr., Ricardo",118.0,L,9.0,4.0,"$50,000","Asmussen, Steven",Winchell Thoroughbreds LLC and Willis Horton ...,4.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,OAKLAWN PARK,Race 11,"April 14, 2018",One And One Eighth,Dirt,THREE YEAR,Cloudy,Fast,Arkansas Derby Grade 1 - Thoroughbred,Tenfold,"Espinoza, Victor",118.0,L,3.0,5.0,"$30,000","Asmussen, Steven",Winchell Thoroughbreds LLC,4.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [97]:
# need baseline model asap, so will drop several columns but keep most informative [intuition]
preps_2018_df = preps_2018_df.drop(columns='Unnamed: 19')

KeyError: "['Unnamed: 19'] not found in axis"

In [98]:
preps_2018_df.head()

,Track Name,Race Number,Date,Track Miles,Surface,Weather,Track Condition,Race,Horse Name,PP,Finish,Money Won,Trainer,Lengths Behind Winner
0,OAKLAWN PARK,Race 11,"April 14, 2018",One And One Eighth,Dirt,Cloudy,Fast,Arkansas Derby Grade 1 - Thoroughbred,Magnum Moon,6.0,1.0,"$600,000","Pletcher, Todd",0.00
1,OAKLAWN PARK,Race 11,"April 14, 2018",One And One Eighth,Dirt,Cloudy,Fast,Arkansas Derby Grade 1 - Thoroughbred,Quip,8.0,2.0,"$200,000","Brisset, Rodolphe",4.00
2,OAKLAWN PARK,Race 11,"April 14, 2018",One And One Eighth,Dirt,Cloudy,Fast,Arkansas Derby Grade 1 - Thoroughbred,Solomini,5.0,3.0,"$100,000","Baffert, Bob",4.25
3,OAKLAWN PARK,Race 11,"April 14, 2018",One And One Eighth,Dirt,Cloudy,Fast,Arkansas Derby Grade 1 - Thoroughbred,Combatant,9.0,4.0,"$50,000","Asmussen, Steven",4.25
4,OAKLAWN PARK,Race 11,"April 14, 2018",One And One Eighth,Dirt,Cloudy,Fast,Arkansas Derby Grade 1 - Thoroughbred,Tenfold,3.0,5.0,"$30,000","Asmussen, Steven",4.50


In [99]:
drop_cols_2018 = ['Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24',
                  'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29',
                  'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34',
                  'Unnamed: 35']

In [100]:
preps_2018_df = preps_2018_df.drop(columns=drop_cols_2018)

KeyError: "['Unnamed: 20' 'Unnamed: 21' 'Unnamed: 22' 'Unnamed: 23' 'Unnamed: 24'\n 'Unnamed: 25' 'Unnamed: 26' 'Unnamed: 27' 'Unnamed: 28' 'Unnamed: 29'\n 'Unnamed: 30' 'Unnamed: 31' 'Unnamed: 32' 'Unnamed: 33' 'Unnamed: 34'\n 'Unnamed: 35'] not found in axis"

In [101]:
preps_2018_df.head()

,Track Name,Race Number,Date,Track Miles,Surface,Weather,Track Condition,Race,Horse Name,PP,Finish,Money Won,Trainer,Lengths Behind Winner
0,OAKLAWN PARK,Race 11,"April 14, 2018",One And One Eighth,Dirt,Cloudy,Fast,Arkansas Derby Grade 1 - Thoroughbred,Magnum Moon,6.0,1.0,"$600,000","Pletcher, Todd",0.00
1,OAKLAWN PARK,Race 11,"April 14, 2018",One And One Eighth,Dirt,Cloudy,Fast,Arkansas Derby Grade 1 - Thoroughbred,Quip,8.0,2.0,"$200,000","Brisset, Rodolphe",4.00
2,OAKLAWN PARK,Race 11,"April 14, 2018",One And One Eighth,Dirt,Cloudy,Fast,Arkansas Derby Grade 1 - Thoroughbred,Solomini,5.0,3.0,"$100,000","Baffert, Bob",4.25
3,OAKLAWN PARK,Race 11,"April 14, 2018",One And One Eighth,Dirt,Cloudy,Fast,Arkansas Derby Grade 1 - Thoroughbred,Combatant,9.0,4.0,"$50,000","Asmussen, Steven",4.25
4,OAKLAWN PARK,Race 11,"April 14, 2018",One And One Eighth,Dirt,Cloudy,Fast,Arkansas Derby Grade 1 - Thoroughbred,Tenfold,3.0,5.0,"$30,000","Asmussen, Steven",4.50


In [102]:
preps_2018_df = preps_2018_df.drop(columns=['Jockey', 'Weight', 'M/E', 'Owner'])

KeyError: "['Jockey' 'Weight' 'M/E' 'Owner'] not found in axis"

In [25]:
preps_2018_df.head()

,Track Name,Race Number,Date,Track Miles,Surface,Age,Weather,Track Condition,Race,Horse Name,PP,Finish,Money Won,Trainer,Lengths Behind Winner
0,OAKLAWN PARK,Race 11,"April 14, 2018",One And One Eighth,Dirt,THREE YEAR,Cloudy,Fast,Arkansas Derby Grade 1 - Thoroughbred,Magnum Moon,6.0,1.0,"$600,000","Pletcher, Todd",0.00
1,OAKLAWN PARK,Race 11,"April 14, 2018",One And One Eighth,Dirt,THREE YEAR,Cloudy,Fast,Arkansas Derby Grade 1 - Thoroughbred,Quip,8.0,2.0,"$200,000","Brisset, Rodolphe",4.00
2,OAKLAWN PARK,Race 11,"April 14, 2018",One And One Eighth,Dirt,THREE YEAR,Cloudy,Fast,Arkansas Derby Grade 1 - Thoroughbred,Solomini,5.0,3.0,"$100,000","Baffert, Bob",4.25
3,OAKLAWN PARK,Race 11,"April 14, 2018",One And One Eighth,Dirt,THREE YEAR,Cloudy,Fast,Arkansas Derby Grade 1 - Thoroughbred,Combatant,9.0,4.0,"$50,000","Asmussen, Steven",4.25
4,OAKLAWN PARK,Race 11,"April 14, 2018",One And One Eighth,Dirt,THREE YEAR,Cloudy,Fast,Arkansas Derby Grade 1 - Thoroughbred,Tenfold,3.0,5.0,"$30,000","Asmussen, Steven",4.50


In [103]:
preps_2018_df.columns

Index(['Track Name', 'Race Number', 'Date', 'Track Miles', 'Surface',
       'Weather', 'Track Condition', 'Race', 'Horse Name', 'PP', 'Finish',
       'Money Won', 'Trainer', 'Lengths Behind Winner'],
      dtype='object')

In [104]:
preps_2018_df['Horse Name'].unique()

array(['Magnum Moon ', 'Quip ', 'Solomini ', 'Combatant ', 'Tenfold ',
       'Good Magic ', 'Flameaway ', 'DQ-Sporting Chance ',
       'Free Drop Billy ', 'Blended Citizen ', 'Paved ', 'City Plan ',
       'Mugaritz ', 'Choo Choo ', 'Audible ', 'Hofburg ', 'Mississippi ',
       'Catholic Boy ', 'Millionaire Runner ', 'Promises Fulfilled ',
       'Strike Power ', 'Machismo ', 'Marconi ', 'Enticed ',
       'Old Time Revival ', 'Firenze Fire ', 'Whereshetoldmetogo ',
       'Tiz Mischief ', 'Pony Up ', 'Seven Trumpets ',
       'Coltandmississippi ', 'Regalian ', 'Factor This ', 'Noble Indy ',
       'Lone Sailor ', 'My Boy Jack ', 'Givemeaminit ',
       'Snapper Sinclair ', 'Instilled Regard ', 'Principe Guilherme ',
       'Zing Zang ', 'Kowboy Karma ', 'Telekinesis ', 'Greyvitos ',
       'Lombo ', 'Ayacara ', 'Dark Vader ', 'Regulate ', 'Peace ',
       'Title Ready ', 'Sporting Chance ', 'Bravazo ', 'High North ',
       'Justify ', "Bolt d'Oro ", 'Core Beliefs ', 'Pepe Tono ',

In [105]:
preps_2018_df.Age.value_counts()

AttributeError: 'DataFrame' object has no attribute 'Age'

In [29]:
preps_2018_df = preps_2018_df.drop(columns='Age')

In [30]:
preps_2018_df.head()

,Track Name,Race Number,Date,Track Miles,Surface,Weather,Track Condition,Race,Horse Name,PP,Finish,Money Won,Trainer,Lengths Behind Winner
0,OAKLAWN PARK,Race 11,"April 14, 2018",One And One Eighth,Dirt,Cloudy,Fast,Arkansas Derby Grade 1 - Thoroughbred,Magnum Moon,6.0,1.0,"$600,000","Pletcher, Todd",0.00
1,OAKLAWN PARK,Race 11,"April 14, 2018",One And One Eighth,Dirt,Cloudy,Fast,Arkansas Derby Grade 1 - Thoroughbred,Quip,8.0,2.0,"$200,000","Brisset, Rodolphe",4.00
2,OAKLAWN PARK,Race 11,"April 14, 2018",One And One Eighth,Dirt,Cloudy,Fast,Arkansas Derby Grade 1 - Thoroughbred,Solomini,5.0,3.0,"$100,000","Baffert, Bob",4.25
3,OAKLAWN PARK,Race 11,"April 14, 2018",One And One Eighth,Dirt,Cloudy,Fast,Arkansas Derby Grade 1 - Thoroughbred,Combatant,9.0,4.0,"$50,000","Asmussen, Steven",4.25
4,OAKLAWN PARK,Race 11,"April 14, 2018",One And One Eighth,Dirt,Cloudy,Fast,Arkansas Derby Grade 1 - Thoroughbred,Tenfold,3.0,5.0,"$30,000","Asmussen, Steven",4.50


In [106]:
preps_2018_df['Track Condition'].value_counts()

Fast              115
Muddy (Sealed)     10
Name: Track Condition, dtype: int64

In [32]:
preps_2018_df['Track Condition'].unique()

array([' Fast', 'Fast', 'Muddy (Sealed)', ' Muddy (Sealed)', nan],
      dtype=object)

In [33]:
preps_2018_df['Track Condition'] = preps_2018_df['Track Condition'].replace({' Fast': 'Fast',
                                                                             ' Muddy (Sealed)':
                                                                             'Muddy (Sealed)'})

In [34]:
preps_2018_df['Track Condition'].unique()

array(['Fast', 'Muddy (Sealed)', nan], dtype=object)

In [107]:
preps_2018_df['Track Condition'].isna().sum()

0

In [108]:
preps_2018_df['Track Condition'].value_counts()

Fast              115
Muddy (Sealed)     10
Name: Track Condition, dtype: int64

In [109]:
preps_2018_df = preps_2018_df.drop(preps_2018_df.index[[223,224,225,226,227,228,229,
                                                     230,231,232]])
preps_2018_df.tail()

IndexError: index 223 is out of bounds for axis 0 with size 125

In [110]:
preps_2018_df.shape

(125, 14)

In [111]:
preps_2018_df['Track Name'].unique()

array(['OAKLAWN PARK ', 'KEENELAND ', 'GOLDEN GATE FIELDS ',
       'GULFSTREAM PARK ', 'AQUEDUCT ', 'FAIR GROUNDS ',
       'SANTA ANITA PARK ', 'TAMPA BAY DOWNS ', 'TURFWAY PARK ',
       'SUNLAND PARK '], dtype=object)

In [112]:
preps_2018_df = preps_2018_df[preps_2018_df.Finish < 6.0]

In [113]:
preps_2018_df.Date.unique()
len(preps_2018_df.Date.unique())

15

In [114]:
date_mapping_2018 = {' April 14, 2018 ': 89, ' April 7, 2018 ': 82, ' February 17, 2018 ': 33,
       ' March 31, 2018 ': 75, ' March 3, 2018 ': 47, ' March 10, 2018 ': 54,
       ' February 3, 2018 ': 19, ' January 13, 2018 ': 13, ' March 24, 2018 ': 68,
       ' March 17, 2018 ': 61, ' February 10, 2018 ': 26, ' January 6, 2018 ': 6,
       ' January 15, 2018 ': 15, ' February 19, 2018 ': 35, ' March 25, 2018 ': 69}

In [115]:
horses_master_2018_df = preps_2018_df.copy()

In [116]:
horses_master_2018_df.head()

,Track Name,Race Number,Date,Track Miles,Surface,Weather,Track Condition,Race,Horse Name,PP,Finish,Money Won,Trainer,Lengths Behind Winner
0,OAKLAWN PARK,Race 11,"April 14, 2018",One And One Eighth,Dirt,Cloudy,Fast,Arkansas Derby Grade 1 - Thoroughbred,Magnum Moon,6.0,1.0,"$600,000","Pletcher, Todd",0.00
1,OAKLAWN PARK,Race 11,"April 14, 2018",One And One Eighth,Dirt,Cloudy,Fast,Arkansas Derby Grade 1 - Thoroughbred,Quip,8.0,2.0,"$200,000","Brisset, Rodolphe",4.00
2,OAKLAWN PARK,Race 11,"April 14, 2018",One And One Eighth,Dirt,Cloudy,Fast,Arkansas Derby Grade 1 - Thoroughbred,Solomini,5.0,3.0,"$100,000","Baffert, Bob",4.25
3,OAKLAWN PARK,Race 11,"April 14, 2018",One And One Eighth,Dirt,Cloudy,Fast,Arkansas Derby Grade 1 - Thoroughbred,Combatant,9.0,4.0,"$50,000","Asmussen, Steven",4.25
4,OAKLAWN PARK,Race 11,"April 14, 2018",One And One Eighth,Dirt,Cloudy,Fast,Arkansas Derby Grade 1 - Thoroughbred,Tenfold,3.0,5.0,"$30,000","Asmussen, Steven",4.50


In [117]:
horses_master_2018_df.Date = horses_master_2018_df.Date.replace(date_mapping_2018)

In [118]:
horses_master_2018_df.head()

,Track Name,Race Number,Date,Track Miles,Surface,Weather,Track Condition,Race,Horse Name,PP,Finish,Money Won,Trainer,Lengths Behind Winner
0,OAKLAWN PARK,Race 11,89,One And One Eighth,Dirt,Cloudy,Fast,Arkansas Derby Grade 1 - Thoroughbred,Magnum Moon,6.0,1.0,"$600,000","Pletcher, Todd",0.00
1,OAKLAWN PARK,Race 11,89,One And One Eighth,Dirt,Cloudy,Fast,Arkansas Derby Grade 1 - Thoroughbred,Quip,8.0,2.0,"$200,000","Brisset, Rodolphe",4.00
2,OAKLAWN PARK,Race 11,89,One And One Eighth,Dirt,Cloudy,Fast,Arkansas Derby Grade 1 - Thoroughbred,Solomini,5.0,3.0,"$100,000","Baffert, Bob",4.25
3,OAKLAWN PARK,Race 11,89,One And One Eighth,Dirt,Cloudy,Fast,Arkansas Derby Grade 1 - Thoroughbred,Combatant,9.0,4.0,"$50,000","Asmussen, Steven",4.25
4,OAKLAWN PARK,Race 11,89,One And One Eighth,Dirt,Cloudy,Fast,Arkansas Derby Grade 1 - Thoroughbred,Tenfold,3.0,5.0,"$30,000","Asmussen, Steven",4.50


In [119]:
horses_master_2018_df = horses_master_2018_df.sort_values(['Date', 'Race'], ascending=[True, True])

In [120]:
horses_master_2018_df.head()
# horses_master_2018_df.shape  # (125, 14)

,Track Name,Race Number,Date,Track Miles,Surface,Weather,Track Condition,Race,Horse Name,PP,Finish,Money Won,Trainer,Lengths Behind Winner
155,SANTA ANITA PARK,Race 5,6,One,Dirt,Cloudy,Fast,Sham S. Grade 3 - Thoroughbred,McKinzie,6.0,1.0,"$60,000","Baffert, Bob",0.00
156,SANTA ANITA PARK,Race 5,6,One,Dirt,Cloudy,Fast,Sham S. Grade 3 - Thoroughbred,All Out Blitz,3.0,2.0,"$20,000","Callaghan, Simon",3.50
157,SANTA ANITA PARK,Race 5,6,One,Dirt,Cloudy,Fast,Sham S. Grade 3 - Thoroughbred,My Boy Jack,1.0,3.0,"$12,000","Desormeaux, J.",7.50
158,SANTA ANITA PARK,Race 5,6,One,Dirt,Cloudy,Fast,Sham S. Grade 3 - Thoroughbred,Shivermetimbers,5.0,4.0,"$6,000","Hollendorfer, Jerry",7.75
159,SANTA ANITA PARK,Race 5,6,One,Dirt,Cloudy,Fast,Sham S. Grade 3 - Thoroughbred,City Plan,7.0,5.0,"$2,000","Harty, Eoin",14.75


In [121]:
len(horses_master_2018_df['Horse Name'].unique())

78

In [122]:
# send it to a csv to quickly populate fields in--gasp--Excel

horses_master_2018_excel = horses_master_2018_df.to_csv('horses-master-2018.csv')

In [123]:
alpha_change_horses_master_2018_df = pd.read_csv('horses-master-2018.csv')

In [124]:
alpha_change_horses_master_2018_df.head()

,Unnamed: 0,Track Name,Race Number,Date,Track Miles,Surface,Weather,Track Condition,Race,Horse Name,PP,Finish,Money Won,Trainer,Lengths Behind Winner
0,155,SANTA ANITA PARK,Race 5,6,One,Dirt,Cloudy,Fast,Sham S. Grade 3 - Thoroughbred,McKinzie,6.0,1.0,"$60,000","Baffert, Bob",0.00
1,156,SANTA ANITA PARK,Race 5,6,One,Dirt,Cloudy,Fast,Sham S. Grade 3 - Thoroughbred,All Out Blitz,3.0,2.0,"$20,000","Callaghan, Simon",3.50
2,157,SANTA ANITA PARK,Race 5,6,One,Dirt,Cloudy,Fast,Sham S. Grade 3 - Thoroughbred,My Boy Jack,1.0,3.0,"$12,000","Desormeaux, J.",7.50
3,158,SANTA ANITA PARK,Race 5,6,One,Dirt,Cloudy,Fast,Sham S. Grade 3 - Thoroughbred,Shivermetimbers,5.0,4.0,"$6,000","Hollendorfer, Jerry",7.75
4,159,SANTA ANITA PARK,Race 5,6,One,Dirt,Cloudy,Fast,Sham S. Grade 3 - Thoroughbred,City Plan,7.0,5.0,"$2,000","Harty, Eoin",14.75


In [130]:
alpha_change_horses_master_2018_df['Money Won'] /= 100

In [54]:
alpha_change_horses_master_2018_df.head()

,Unnamed: 0,Track Name,Race Number,Date,Track Miles,Surface,Weather,Track Condition,Race,Horse Name,PP,Finish,Money Won,Trainer,Lengths Behind Winner
0,155,SANTA ANITA PARK,Race 5,6,One,Dirt,Cloudy,Fast,Sham S. Grade 3 - Thoroughbred,McKinzie,6.0,1.0,"$60,000","Baffert, Bob",0.00
1,156,SANTA ANITA PARK,Race 5,6,One,Dirt,Cloudy,Fast,Sham S. Grade 3 - Thoroughbred,All Out Blitz,3.0,2.0,"$20,000","Callaghan, Simon",3.50
2,157,SANTA ANITA PARK,Race 5,6,One,Dirt,Cloudy,Fast,Sham S. Grade 3 - Thoroughbred,My Boy Jack,1.0,3.0,"$12,000","Desormeaux, J.",7.50
3,158,SANTA ANITA PARK,Race 5,6,One,Dirt,Cloudy,Fast,Sham S. Grade 3 - Thoroughbred,Shivermetimbers,5.0,4.0,"$6,000","Hollendorfer, Jerry",7.75
4,159,SANTA ANITA PARK,Race 5,6,One,Dirt,Cloudy,Fast,Sham S. Grade 3 - Thoroughbred,City Plan,7.0,5.0,"$2,000","Harty, Eoin",14.75


In [125]:
alpha_change_horses_master_2018_df = alpha_change_horses_master_2018_df.drop(columns=['Unnamed: 0',
                                                                                      'Race Number'])

In [126]:
alpha_change_horses_master_2018_df.head()

,Track Name,Date,Track Miles,Surface,Weather,Track Condition,Race,Horse Name,PP,Finish,Money Won,Trainer,Lengths Behind Winner
0,SANTA ANITA PARK,6,One,Dirt,Cloudy,Fast,Sham S. Grade 3 - Thoroughbred,McKinzie,6.0,1.0,"$60,000","Baffert, Bob",0.00
1,SANTA ANITA PARK,6,One,Dirt,Cloudy,Fast,Sham S. Grade 3 - Thoroughbred,All Out Blitz,3.0,2.0,"$20,000","Callaghan, Simon",3.50
2,SANTA ANITA PARK,6,One,Dirt,Cloudy,Fast,Sham S. Grade 3 - Thoroughbred,My Boy Jack,1.0,3.0,"$12,000","Desormeaux, J.",7.50
3,SANTA ANITA PARK,6,One,Dirt,Cloudy,Fast,Sham S. Grade 3 - Thoroughbred,Shivermetimbers,5.0,4.0,"$6,000","Hollendorfer, Jerry",7.75
4,SANTA ANITA PARK,6,One,Dirt,Cloudy,Fast,Sham S. Grade 3 - Thoroughbred,City Plan,7.0,5.0,"$2,000","Harty, Eoin",14.75


In [127]:
alpha_change_horses_master_2018_df = alpha_change_horses_master_2018_df.drop(columns='Track Miles')

In [128]:
alpha_change_horses_master_2018_df.Weather.unique()

array([' Cloudy', 'Clear', 'Cloudy', ' Clear', ' Showery', 'Showery'],
      dtype=object)

In [129]:
alpha_change_horses_master_2018_df.Weather = alpha_change_horses_master_2018_df.Weather \
                                             .replace({' Cloudy': 'Cloudy',
                                                       ' Clear': 'Clear',
                                                       ' Showery': 'Showery'})

In [130]:
alpha_change_horses_master_2018_df.Weather.unique()

array(['Cloudy', 'Clear', 'Showery'], dtype=object)

In [131]:
alpha_change_horses_master_2018_df.Weather = alpha_change_horses_master_2018_df.Weather \
                                             .replace({'Cloudy': 2, 'Clear': 1, 'Showery': 2})

In [132]:
alpha_change_horses_master_2018_df.head()

,Track Name,Date,Surface,Weather,Track Condition,Race,Horse Name,PP,Finish,Money Won,Trainer,Lengths Behind Winner
0,SANTA ANITA PARK,6,Dirt,2,Fast,Sham S. Grade 3 - Thoroughbred,McKinzie,6.0,1.0,"$60,000","Baffert, Bob",0.00
1,SANTA ANITA PARK,6,Dirt,2,Fast,Sham S. Grade 3 - Thoroughbred,All Out Blitz,3.0,2.0,"$20,000","Callaghan, Simon",3.50
2,SANTA ANITA PARK,6,Dirt,2,Fast,Sham S. Grade 3 - Thoroughbred,My Boy Jack,1.0,3.0,"$12,000","Desormeaux, J.",7.50
3,SANTA ANITA PARK,6,Dirt,2,Fast,Sham S. Grade 3 - Thoroughbred,Shivermetimbers,5.0,4.0,"$6,000","Hollendorfer, Jerry",7.75
4,SANTA ANITA PARK,6,Dirt,2,Fast,Sham S. Grade 3 - Thoroughbred,City Plan,7.0,5.0,"$2,000","Harty, Eoin",14.75


In [133]:
alpha_change_horses_master_2018_df['Track Condition'].unique()

array(['Fast', 'Muddy (Sealed)'], dtype=object)

In [134]:
alpha_change_horses_master_2018_df['Track Condition'] = alpha_change_horses_master_2018_df \
                                                        ['Track Condition'].replace({'Fast': 1,
                                                                                     'Muddy (Sealed)': 2})

In [135]:
alpha_change_horses_master_2018_df.head()

,Track Name,Date,Surface,Weather,Track Condition,Race,Horse Name,PP,Finish,Money Won,Trainer,Lengths Behind Winner
0,SANTA ANITA PARK,6,Dirt,2,1,Sham S. Grade 3 - Thoroughbred,McKinzie,6.0,1.0,"$60,000","Baffert, Bob",0.00
1,SANTA ANITA PARK,6,Dirt,2,1,Sham S. Grade 3 - Thoroughbred,All Out Blitz,3.0,2.0,"$20,000","Callaghan, Simon",3.50
2,SANTA ANITA PARK,6,Dirt,2,1,Sham S. Grade 3 - Thoroughbred,My Boy Jack,1.0,3.0,"$12,000","Desormeaux, J.",7.50
3,SANTA ANITA PARK,6,Dirt,2,1,Sham S. Grade 3 - Thoroughbred,Shivermetimbers,5.0,4.0,"$6,000","Hollendorfer, Jerry",7.75
4,SANTA ANITA PARK,6,Dirt,2,1,Sham S. Grade 3 - Thoroughbred,City Plan,7.0,5.0,"$2,000","Harty, Eoin",14.75


In [136]:
len(alpha_change_horses_master_2018_df.Race.unique())

25

In [137]:
race_ordinals = []

for num in range(1, 26):
    for n in range(5):
        race_ordinals.append(num)
race_ordinals[:16]

[1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4]

In [138]:
alpha_change_horses_master_2018_df['Race Ordinal'] = pd.Series(race_ordinals)

In [139]:
alpha_change_horses_master_2018_df.head(10)

,Track Name,Date,Surface,Weather,Track Condition,Race,Horse Name,PP,Finish,Money Won,Trainer,Lengths Behind Winner,Race Ordinal
0,SANTA ANITA PARK,6,Dirt,2,1,Sham S. Grade 3 - Thoroughbred,McKinzie,6.0,1.0,"$60,000","Baffert, Bob",0.00,1
1,SANTA ANITA PARK,6,Dirt,2,1,Sham S. Grade 3 - Thoroughbred,All Out Blitz,3.0,2.0,"$20,000","Callaghan, Simon",3.50,1
2,SANTA ANITA PARK,6,Dirt,2,1,Sham S. Grade 3 - Thoroughbred,My Boy Jack,1.0,3.0,"$12,000","Desormeaux, J.",7.50,1
3,SANTA ANITA PARK,6,Dirt,2,1,Sham S. Grade 3 - Thoroughbred,Shivermetimbers,5.0,4.0,"$6,000","Hollendorfer, Jerry",7.75,1
4,SANTA ANITA PARK,6,Dirt,2,1,Sham S. Grade 3 - Thoroughbred,City Plan,7.0,5.0,"$2,000","Harty, Eoin",14.75,1
5,AQUEDUCT,13,Dirt,1,2,Jerome S. Listed - Thoroughbred,Firenze Fire,7.0,1.0,"$90,000","Servis, Jason",0.00,2
6,AQUEDUCT,13,Dirt,1,2,Jerome S. Listed - Thoroughbred,Seven Trumpets,2.0,2.0,"$30,000","Romans, Dale",0.50,2
7,AQUEDUCT,13,Dirt,1,2,Jerome S. Listed - Thoroughbred,Coltandmississippi,1.0,3.0,"$15,000","Pletcher, Todd",5.75,2
8,AQUEDUCT,13,Dirt,1,2,Jerome S. Listed - Thoroughbred,Regalian,3.0,4.0,"$7,500","Englehart, Chris",6.25,2
9,AQUEDUCT,13,Dirt,1,2,Jerome S. Listed - Thoroughbred,Factor This,4.0,5.0,"$4,500","Barker, Edward",10.50,2


In [140]:
kd_2019_contenders_df.Trainer.value_counts()

Bob Baffert          3
William Mott         2
Todd Pletcher        2
Kiaran McLaughlin    1
Richard Mandella     1
Bret Calhoun         1
Koichi Tsunoda       1
Michael Trombetta    1
Jason Servis         1
Steve Asmussen       1
Danny Gargan         1
Peter Miller         1
George Weaver        1
Shug McGaughey       1
Mark Casse           1
Brendan Walsh        1
Name: Trainer, dtype: int64

In [141]:
alpha_change_horses_master_2018_df.Trainer.unique()
# based on lines just above and below, will rank Bob Baffert first; Todd Pletcher,
# William Mott, and Steve Asmussen all second, and all other trainers third

array([' Baffert, Bob', ' Callaghan, Simon', ' Desormeaux, J.',
       ' Hollendorfer, Jerry', ' Harty, Eoin', ' Servis, Jason',
       ' Romans, Dale', ' Pletcher, Todd', ' Englehart, Chris',
       ' Barker, Edward', ' Asmussen, Steven', ' Jones, J.',
       ' Milligan, Allen', ' Frederick, Edward', ' McLaughlin, Kiaran',
       ' Pender, Michael', ' Eurton, Peter', ' Mandella, Richard',
       ' Casse, Mark', ' Thomas, Jonathan', ' Scott, Joan',
       ' McCarthy, Michael', " O'Neill, Doug", ' Wong, Jonathan',
       ' Lukas, D.', ' Cox, Brad', ' Hennig, Mark', ' Brown, Chad',
       ' Quartarolo, Anthony', ' Decker, Kenneth', ' Pecoraro, Anthony',
       ' Brisset, Rodolphe', ' Ruis, Mick', ' Ward, Wesley',
       ' Amoss, Thomas', ' Stewart, Dallas', ' Fincher, Todd',
       ' Mott, William', ' Mejia, Jaime', ' Garcia, Victor',
       " D'Amato, Philip", ' Kitchingman, Adam'], dtype=object)

In [142]:
alpha_change_horses_master_2018_df.Trainer.value_counts()

 Pletcher, Todd         17
 Asmussen, Steven       14
 Romans, Dale           10
 Baffert, Bob            9
 Hollendorfer, Jerry     6
 Desormeaux, J.          6
 Casse, Mark             5
 Servis, Jason           5
 Lukas, D.               4
 McLaughlin, Kiaran      4
 O'Neill, Doug           3
 Mandella, Richard       3
 Eurton, Peter           3
 Brown, Chad             2
 Wong, Jonathan          2
 Brisset, Rodolphe       2
 Harty, Eoin             2
 Ruis, Mick              2
 Amoss, Thomas           2
 Thomas, Jonathan        2
 Stewart, Dallas         1
 Hennig, Mark            1
 Garcia, Victor          1
 Decker, Kenneth         1
 Pender, Michael         1
 Cox, Brad               1
 Kitchingman, Adam       1
 McCarthy, Michael       1
 Frederick, Edward       1
 Barker, Edward          1
 Mott, William           1
 Pecoraro, Anthony       1
 Mejia, Jaime            1
 Jones, J.               1
 D'Amato, Philip         1
 Callaghan, Simon        1
 Scott, Joan             1
 

In [143]:
alpha_change_horses_master_2018_df['Trainer Rank'] = pd.Series([3 for _ in range(125)])

In [144]:
alpha_change_horses_master_2018_df.head()

,Track Name,Date,Surface,Weather,Track Condition,Race,Horse Name,PP,Finish,Money Won,Trainer,Lengths Behind Winner,Race Ordinal,Trainer Rank
0,SANTA ANITA PARK,6,Dirt,2,1,Sham S. Grade 3 - Thoroughbred,McKinzie,6.0,1.0,"$60,000","Baffert, Bob",0.00,1,3
1,SANTA ANITA PARK,6,Dirt,2,1,Sham S. Grade 3 - Thoroughbred,All Out Blitz,3.0,2.0,"$20,000","Callaghan, Simon",3.50,1,3
2,SANTA ANITA PARK,6,Dirt,2,1,Sham S. Grade 3 - Thoroughbred,My Boy Jack,1.0,3.0,"$12,000","Desormeaux, J.",7.50,1,3
3,SANTA ANITA PARK,6,Dirt,2,1,Sham S. Grade 3 - Thoroughbred,Shivermetimbers,5.0,4.0,"$6,000","Hollendorfer, Jerry",7.75,1,3
4,SANTA ANITA PARK,6,Dirt,2,1,Sham S. Grade 3 - Thoroughbred,City Plan,7.0,5.0,"$2,000","Harty, Eoin",14.75,1,3


In [145]:
alpha_change_horses_master_2018_df.loc[(alpha_change_horses_master_2018_df['Trainer'] == ' Baffert, Bob'),
                                       'Trainer Rank'] = 1
# df.loc[(df['A'] == 'blue') & (df['B'] == 'red') & (df['B'] == 'square'),'label'] = 'M5'

In [146]:
# change rank for Pletcher, Mott, and Asmussen each to 2
alpha_change_horses_master_2018_df.loc[(alpha_change_horses_master_2018_df['Trainer'] == ' Pletcher, Todd'),
                                       'Trainer Rank'] = 2
alpha_change_horses_master_2018_df.loc[(alpha_change_horses_master_2018_df['Trainer'] == ' Mott, William'),
                                       'Trainer Rank'] = 2
alpha_change_horses_master_2018_df.loc[(alpha_change_horses_master_2018_df['Trainer'] == ' Asmussen, Steven'),
                                       'Trainer Rank'] = 2

In [147]:
kd_2019_contenders_df.Sire.value_counts()

Candy Ride (ARG)    2
War Front           2
Just a Way          1
Arch                1
Graydar             1
Hard Spun           1
Noble Mission       1
Goldencents         1
Daaher              1
City Zip            1
Lookin At Lucky     1
Take Charge Indy    1
Point of Entry      1
Tapit               1
First Samurai       1
Hat Trick (JPN)     1
New Year's Day      1
Quality Road        1
Name: Sire, dtype: int64

In [148]:
alpha_change_horses_master_2018_df['Winner Sire'].unique()

KeyError: 'Winner Sire'

In [149]:
for sire in kd_2019_contenders_df.Sire.values:
    if sire in alpha_change_horses_master_2018_df['Winner Sire'].unique():
        print(sire)

KeyError: 'Winner Sire'

In [150]:
kd_2019_contenders_df.head()

,Horse,Trainer,Sire,Earnings (K USD)
0,Tacitus,William Mott,Tapit,610
1,Omaha Beach,Richard Mandella,War Front,1050
2,Vekoma,George Weaver,Candy Ride (ARG),747.6
3,Plus Que Parfait,Brendan Walsh,Point of Entry,1540.4
4,Roadster,Bob Baffert,Quality Road,636


In [151]:
# Add 'Winner Sire Rank' with 1 for top sires, 2 for all others
alpha_change_horses_master_2018_df['Sire Rank'] = pd.Series([2 for _ in range(125)])


In [152]:
alpha_change_horses_master_2018_df.head(20)

,Track Name,Date,Surface,Weather,Track Condition,Race,Horse Name,PP,Finish,Money Won,Trainer,Lengths Behind Winner,Race Ordinal,Trainer Rank,Sire Rank
0,SANTA ANITA PARK,6,Dirt,2,1,Sham S. Grade 3 - Thoroughbred,McKinzie,6.0,1.0,"$60,000","Baffert, Bob",0.00,1,1,2
1,SANTA ANITA PARK,6,Dirt,2,1,Sham S. Grade 3 - Thoroughbred,All Out Blitz,3.0,2.0,"$20,000","Callaghan, Simon",3.50,1,3,2
2,SANTA ANITA PARK,6,Dirt,2,1,Sham S. Grade 3 - Thoroughbred,My Boy Jack,1.0,3.0,"$12,000","Desormeaux, J.",7.50,1,3,2
3,SANTA ANITA PARK,6,Dirt,2,1,Sham S. Grade 3 - Thoroughbred,Shivermetimbers,5.0,4.0,"$6,000","Hollendorfer, Jerry",7.75,1,3,2
4,SANTA ANITA PARK,6,Dirt,2,1,Sham S. Grade 3 - Thoroughbred,City Plan,7.0,5.0,"$2,000","Harty, Eoin",14.75,1,3,2
5,AQUEDUCT,13,Dirt,1,2,Jerome S. Listed - Thoroughbred,Firenze Fire,7.0,1.0,"$90,000","Servis, Jason",0.00,2,3,2
6,AQUEDUCT,13,Dirt,1,2,Jerome S. Listed - Thoroughbred,Seven Trumpets,2.0,2.0,"$30,000","Romans, Dale",0.50,2,3,2
7,AQUEDUCT,13,Dirt,1,2,Jerome S. Listed - Thoroughbred,Coltandmississippi,1.0,3.0,"$15,000","Pletcher, Todd",5.75,2,2,2
8,AQUEDUCT,13,Dirt,1,2,Jerome S. Listed - Thoroughbred,Regalian,3.0,4.0,"$7,500","Englehart, Chris",6.25,2,3,2
9,AQUEDUCT,13,Dirt,1,2,Jerome S. Listed - Thoroughbred,Factor This,4.0,5.0,"$4,500","Barker, Edward",10.50,2,3,2


In [153]:
alpha_change_horses_master_2018_df.loc[(alpha_change_horses_master_2018_df['Winner Sire'] == 'Quality Road'),
                                       'Sire Rank'] = 1
alpha_change_horses_master_2018_df.loc[(alpha_change_horses_master_2018_df['Winner Sire'] == 'Arch'),
                                       'Sire Rank'] = 1
alpha_change_horses_master_2018_df.loc[(alpha_change_horses_master_2018_df['Winner Sire'] == 'Take Charge Indy'),
                                       'Sire Rank'] = 1
alpha_change_horses_master_2018_df.loc[(alpha_change_horses_master_2018_df['Winner Sire'] == 'Graydar'),
                                       'Sire Rank'] = 1

KeyError: 'Winner Sire'

In [154]:
alpha_change_horses_master_2018_df.head()

,Track Name,Date,Surface,Weather,Track Condition,Race,Horse Name,PP,Finish,Money Won,Trainer,Lengths Behind Winner,Race Ordinal,Trainer Rank,Sire Rank
0,SANTA ANITA PARK,6,Dirt,2,1,Sham S. Grade 3 - Thoroughbred,McKinzie,6.0,1.0,"$60,000","Baffert, Bob",0.00,1,1,2
1,SANTA ANITA PARK,6,Dirt,2,1,Sham S. Grade 3 - Thoroughbred,All Out Blitz,3.0,2.0,"$20,000","Callaghan, Simon",3.50,1,3,2
2,SANTA ANITA PARK,6,Dirt,2,1,Sham S. Grade 3 - Thoroughbred,My Boy Jack,1.0,3.0,"$12,000","Desormeaux, J.",7.50,1,3,2
3,SANTA ANITA PARK,6,Dirt,2,1,Sham S. Grade 3 - Thoroughbred,Shivermetimbers,5.0,4.0,"$6,000","Hollendorfer, Jerry",7.75,1,3,2
4,SANTA ANITA PARK,6,Dirt,2,1,Sham S. Grade 3 - Thoroughbred,City Plan,7.0,5.0,"$2,000","Harty, Eoin",14.75,1,3,2


In [156]:
alpha_change_horses_master_2018_df['Money Won'] = alpha_change_horses_master_2018_df['Money Won'] \
                                                  .apply(lambda x: x.replace(' ', ''))
alpha_change_horses_master_2018_df['Money Won'] = alpha_change_horses_master_2018_df['Money Won'] \
                                                  .apply(lambda x: x.replace(',', ''))
alpha_change_horses_master_2018_df['Money Won'] = alpha_change_horses_master_2018_df['Money Won'] \
                                                  .apply(lambda x: x.replace('$', ''))
alpha_change_horses_master_2018_df['Money Won'] = alpha_change_horses_master_2018_df['Money Won'] \
                                                  .apply(lambda x: int(x))

In [159]:
alpha_change_horses_master_2018_df['Money Won'] /= 1000

In [157]:
alpha_change_horses_master_2018_df['Kentucky Derby Finish'] = pd.Series([21 for _ in range(125)])

In [160]:
alpha_change_horses_master_2018_df.head()

,Track Name,Date,Surface,Weather,Track Condition,Race,Horse Name,PP,Finish,Money Won,Trainer,Lengths Behind Winner,Race Ordinal,Trainer Rank,Sire Rank,Kentucky Derby Finish
0,SANTA ANITA PARK,6,Dirt,2,1,Sham S. Grade 3 - Thoroughbred,McKinzie,6.0,1.0,60.0,"Baffert, Bob",0.00,1,1,2,21
1,SANTA ANITA PARK,6,Dirt,2,1,Sham S. Grade 3 - Thoroughbred,All Out Blitz,3.0,2.0,20.0,"Callaghan, Simon",3.50,1,3,2,21
2,SANTA ANITA PARK,6,Dirt,2,1,Sham S. Grade 3 - Thoroughbred,My Boy Jack,1.0,3.0,12.0,"Desormeaux, J.",7.50,1,3,2,21
3,SANTA ANITA PARK,6,Dirt,2,1,Sham S. Grade 3 - Thoroughbred,Shivermetimbers,5.0,4.0,6.0,"Hollendorfer, Jerry",7.75,1,3,2,21
4,SANTA ANITA PARK,6,Dirt,2,1,Sham S. Grade 3 - Thoroughbred,City Plan,7.0,5.0,2.0,"Harty, Eoin",14.75,1,3,2,21


In [161]:
# Change 'Kentucky Derby Finish' values for top 5 finishers in that race
alpha_change_horses_master_2018_df.loc[(alpha_change_horses_master_2018_df['Horse Name'] == 'Justify '),
                                       'Kentucky Derby Finish'] = 1

In [162]:
alpha_change_horses_master_2018_df.loc[(alpha_change_horses_master_2018_df['Horse Name'] == 'Good Magic '),
                                       'Kentucky Derby Finish'] = 2
alpha_change_horses_master_2018_df.loc[(alpha_change_horses_master_2018_df['Horse Name'] == 'Audible '),
                                       'Kentucky Derby Finish'] = 3
alpha_change_horses_master_2018_df.loc[(alpha_change_horses_master_2018_df['Horse Name'] == 'Instilled Regard '),
                                       'Kentucky Derby Finish'] = 4
alpha_change_horses_master_2018_df.loc[(alpha_change_horses_master_2018_df['Horse Name'] == 'My Boy Jack '),
                                       'Kentucky Derby Finish'] = 5

In [163]:
alpha_change_horses_master_2018_df.loc[(alpha_change_horses_master_2018_df['Horse Name'] == 'Bravazo '),
                                       'Kentucky Derby Finish'] = 6
alpha_change_horses_master_2018_df.loc[(alpha_change_horses_master_2018_df['Horse Name'] == 'Hofburg '),
                                       'Kentucky Derby Finish'] = 7
alpha_change_horses_master_2018_df.loc[(alpha_change_horses_master_2018_df['Horse Name'] == 'Lone Sailor '),
                                       'Kentucky Derby Finish'] = 8
alpha_change_horses_master_2018_df.loc[(alpha_change_horses_master_2018_df['Horse Name'] == 'Vino Rosso '),
                                       'Kentucky Derby Finish'] = 9
alpha_change_horses_master_2018_df.loc[(alpha_change_horses_master_2018_df['Horse Name'] == 'Solomini '),
                                       'Kentucky Derby Finish'] = 10
alpha_change_horses_master_2018_df.loc[(alpha_change_horses_master_2018_df['Horse Name'] == 'Firenze Fire '),
                                       'Kentucky Derby Finish'] = 11
alpha_change_horses_master_2018_df.loc[(alpha_change_horses_master_2018_df['Horse Name'] == 'Bolt d\'Oro '),
                                       'Kentucky Derby Finish'] = 12
alpha_change_horses_master_2018_df.loc[(alpha_change_horses_master_2018_df['Horse Name'] == 'Flameaway '),
                                       'Kentucky Derby Finish'] = 13
alpha_change_horses_master_2018_df.loc[(alpha_change_horses_master_2018_df['Horse Name'] == 'Enticed '),
                                       'Kentucky Derby Finish'] = 14

In [164]:
alpha_change_horses_master_2018_df.loc[(alpha_change_horses_master_2018_df['Horse Name'] == 'Promises Fulfilled '),
                                       'Kentucky Derby Finish'] = 15
alpha_change_horses_master_2018_df.loc[(alpha_change_horses_master_2018_df['Horse Name'] == 'Free Drop Billy '),
                                       'Kentucky Derby Finish'] = 16
alpha_change_horses_master_2018_df.loc[(alpha_change_horses_master_2018_df['Horse Name'] == 'Noble Indy '),
                                       'Kentucky Derby Finish'] = 17
alpha_change_horses_master_2018_df.loc[(alpha_change_horses_master_2018_df['Horse Name'] == 'Combatant '),
                                       'Kentucky Derby Finish'] = 18
alpha_change_horses_master_2018_df.loc[(alpha_change_horses_master_2018_df['Horse Name'] == 'Magnum Moon '),
                                       'Kentucky Derby Finish'] = 19
alpha_change_horses_master_2018_df.loc[(alpha_change_horses_master_2018_df['Horse Name'] == 'Mendelssohn '),
                                       'Kentucky Derby Finish'] = 20

In [165]:
alpha_change_horses_master_2018_df['Kentucky Derby Finish'].value_counts()

21    81
18     4
11     4
5      4
9      3
16     3
14     3
13     3
4      3
2      2
3      2
10     2
19     2
12     2
17     2
8      1
7      1
6      1
15     1
1      1
Name: Kentucky Derby Finish, dtype: int64

In [166]:
alpha_change_horses_master_2018_df = alpha_change_horses_master_2018_df.drop(columns='Horse Ordinal')
alpha_change_horses_master_2018_df.head()

KeyError: "['Horse Ordinal'] not found in axis"

In [167]:
## Now to see about modeling the data
X = alpha_change_horses_master_2018_df[['Date', 'Race Ordinal', 'Weather', 'Track Condition',
                                        'Finish', 'PP', 'Sire Rank', 'Trainer Rank',
                                        'Lengths Behind Winner', 'Money Won']]
y = alpha_change_horses_master_2018_df['Kentucky Derby Finish']

In [168]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [169]:
from sklearn.ensemble import RandomForestClassifier

In [170]:
rfc = RandomForestClassifier(n_estimators=100)

In [171]:
rfc.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [172]:
rfc.score(X_test, y_test)

0.72

In [173]:
### bringing back down 2019 KD contender arrays and updating for use in model
# From most recent best performance, need:
# 'Date', 'Race Ordinal', 'Weather', 'Track Condition', 'Finish', 'PP', 'Sire Rank', 'Trainer Rank',
# 'Lengths Behind Winner', 'Money Won'

k_tacitus = [96, 22, 1, 1, 1, 1, 2, 2, 0.0, 40.0]  # won at Wood, followed by Tax, Haikal; 2W's; Tapit, Mott
k_omaha_beach = [103, 25, 3, 2, 1, 1, 1, 3, 0.0, 60.0]  # won @ Arkansas vs. Improb, C. House; 2W's; War Front, Mandella
k_vekoma = [96, 23, 1, 1, 1, 3, 1, 3, 0.0, 60.0]  # won at Bluegrass vs. Win Win Win; Candy Ride (ARG), Weaver
k_plus_que_parfait = [89, 20, 1, 1, 1, 4, 2, 3, 0.0, 150.0]  # won at UAE; 0 other results; Point of Entry, Walsh
k_roadster = [96, 24, 1, 1, 1, 4, 1, 1, 0.0, 60.0]  # won at Santa Anita; no other results; Quality Road, Baffert
k_by_my_standards = [82, 18, 1, 1, 1, 4, 2, 3, 0.0, 100.0]  # won at Louisiana; 0 other results; Goldencents, Calhoun
k_maximum_security = [89, 21, 1, 1, 1, 4, 2, 3, 0.0, 58.28]  # won at Florida; 0 other results; New Year's Day, Servis
k_game_winner = [96, 24, 1, 1, 2, 2, 1, 1, 0.5, 20.0]  # 2nd at Santa Anita; Candy Ride (ARG), Baffert
k_code_of_honor = [61, 12, 1, 1, 1, 4, 2, 3, 0.0, 23.312]  # won at Fountain of Youth; Noble Mission, McGaughey
k_haikal = [68, 13, 1, 1, 1, 4, 2, 3, 0.0, 16.5]  # won at Gotham; Daaher, McLaughlin
k_improbable = [103, 25, 3, 2, 2, 2, 2, 1, 1.0, 20.0]  # 2nd at Arkansas; City Zip, Baffert
k_war_of_will = [47, 10, 2, 1, 1, 1, 1, 3, 0.0, 24.0]  # won at Risen Star; War Front, Casse
k_long_range_toddy = [75, 16, 1, 1, 1, 3, 1, 2, 0.0, 45.0]  # won at Reb 1; Take Charge Indy, Asmussen
k_tax = [33, 5, 1, 1, 1, 4, 1, 3, 0.0, 13.75]  # won at Withers; Arch, Gargan
k_cutting_humor = [83, 19, 1, 1, 1, 4, 2, 2, 0.0, 46.08]  # won at Sunland; 0 other results; First Samurai, Pletcher
k_win_win_win = [96, 23, 1, 1, 2, 3, 2, 3, 0.25, 20.0]  # 2nd at Bluegrass; Hat Trick (JPN), Trombetta
k_country_house = [47, 10, 2, 1, 2, 4, 2, 2, 2.25, 8.0]  # 2nd at Risen Star; Lookin At Lucky, Mott
k_gray_magician = [89, 20, 1, 1, 2, 4, 1, 3, 0.75, 50.0]  # 2nd at UAE; Graydar, Miller
k_spinoff = [82, 18, 1, 1, 2, 4, 2, 2, 0.75, 60.0]  # 2nd at Louisiana; no other results; Hard Spun, Pletcher
k_master_fencer_jpn = [103, 27, 1, 1, 1, 4, 2, 3, 0.0, 20.0]  # via Japan Stakes races; Just a Way, K. Tsunoda
# numbers entered for master_fencer_jpn are arbitrary due to not having data immediately available

In [174]:
kd_2019_contenders_final = [k_tacitus, k_omaha_beach, k_vekoma, k_plus_que_parfait, k_roadster,
                            k_by_my_standards, k_maximum_security, k_game_winner, k_code_of_honor,
                            k_haikal, k_improbable, k_war_of_will, k_long_range_toddy, k_tax,
                            k_cutting_humor, k_win_win_win, k_country_house, k_gray_magician,
                            k_spinoff, k_master_fencer_jpn]

In [185]:
rfc.predict(np.array(kd_2019_contenders_final))

array([21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21,
       21, 21, 21], dtype=int64)

In [186]:
len(rfc.predict(kd_2019_contenders_final))

20